In [4]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [5]:
FILE_BASE_URL_TRAIN_QUESTIONS = "LR1_dev.csv"
FILE_BASE_URL_TRAIN_ANSWERS = "LR1_dev_answers.csv"
FILE_BASE_URL_TEST_QUESTIONS = "LR1.csv"

In [7]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.expand_frame_repr', False)

In [8]:
train_df_questions = pd.read_csv(
    FILE_BASE_URL_TRAIN_QUESTIONS
)
train_df_questions.columns = [
    "id",
    "question",
    "options",
    "category"
]

train_df_answers = pd.read_csv(
    FILE_BASE_URL_TRAIN_ANSWERS
)
train_df_answers.columns = [
    "answer_id",
    "answer"
]

train_df = pd.concat(
    (train_df_questions,
    train_df_answers),
    axis = 1
)

train_df = train_df[["id", "question", "options", "category", "answer"]]
train_df.category.value_counts()

category
physics             6
other               5
biology             5
economics           5
chemistry           5
math                4
health              4
psychology          4
history             3
law                 3
computer science    2
engineering         2
business            1
philosophy          1
Name: count, dtype: int64

In [9]:
test_df_questions = pd.read_csv(
    FILE_BASE_URL_TEST_QUESTIONS
)

test_df_questions.columns = [
    "id",
    "question",
    "options",
    "category"
]

test_df_questions.category.value_counts()


category
math                13
physics             11
psychology          10
other                9
law                  8
business             7
biology              7
engineering          7
economics            6
chemistry            6
philosophy           5
health               5
computer science     4
history              2
Name: count, dtype: int64

In [ ]:
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import re
import os
import gc
import json

/Users/karlkorhonen/проекты/Data_analysis/LLM/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import re


def options_parser(
    options
):
    if isinstance(options, list):
        return options
    if isinstance(options, str):
        try:
            return json.loads(options)
        except Exception as e:
            final = []
            if options.startswith('[') and options.endswith(']'):
                options = options.lstrip("[").rstrip("]")
                single_quote = r"'(.*?)'(?=\s*(?:'|$))"
                double_quote = r'"(.*?)"(?=\s*(?:"|$))'
                single = re.findall(single_quote, options, re.DOTALL)
                if not single:
                    single = re.findall(double_quote, options, re.DOTALL)
                for phrase in single:
                    clean = phrase.replace("\\'", "'").replace("\\n", "\n").replace('\\"', '"').strip()
                    final.append(clean)
                return final
            if not final:
                if "," in options:
                    final = list(map(str.strip, options.split(",")))
                    return final
        return options


In [9]:
print(options_parser("['один'\n'два']"))

['один', 'два']


In [ ]:
def load_model_tokenizer(
    model_name = "Qwen/Qwen2.5-14B-Instruct",
):
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    if tokenizer.pad_token is None or not tokenizer.pad_token:
        tokenizer.pad_token = tokenizer.eos_token
    quantization_config = BitsAndBytesConfig(
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        load_in_4bit=True,
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quantization_config,
        device_map="auto",
        trust_remote_code=True
    )
    print(f"✅ Модель {model_name} успешно загружена!")
    return model, tokenizer

model_14b, tokenizer_14b = load_model_tokenizer("Qwen/Qwen2.5-14B-Instruct")


In [2]:
import pandas as pd
import os
from pathlib import Path
from typing import Optional, Dict, Any
import time
from tqdm import tqdm

In [ ]:
PROMPTS = {
    "physics": """
Ты эксперт по физике. Реши задачу по физике, используя метод пошагового рассуждения (Chain-of-Thought):

1. Сначала определи, какой раздел физики задействован (механика, электромагнетизм, термодинамика, оптика и т.д.)
2. Выпиши известные величины и что нужно найти
3. Вспомни соответствующие формулы и физические принципы
4. Примени логическое рассуждение по шагам, показывая все вычисления
5. Проверь единицы измерения и размерность
6. Оцени, является ли ответ физически осмысленным

Внимание: После завершения рассуждения дай ТОЛЬКО номер правильного варианта (0, 1, 2 или 3). Не пиши объяснений.
""",

    "chemistry": """
Ты эксперт-химик. Ответь на вопрос по химии, используя методику пошагового анализа:

1. Определи тип химической задачи: стехиометрия, химическая связь, термохимия, кинетика, равновесие и т.д.
2. Если есть химическое уравнение - проверь его баланс
3. Определи молярные массы и стехиометрические соотношения
4. Для вопросов о свойствах - вспомни периодические закономерности
5. Для органической химии - анализируй функциональные группы
6. Проверь соответствие законов сохранения

После анализа дай ТОЛЬКО номер правильного ответа (0-3). Без дополнительных объяснений.
""",

    "biology": """
Ты биолог с глубокими знаниями. Используй метод логической цепочки для ответа на вопрос по биологии:

1. Определи область биологии: молекулярная, клеточная, генетика, эволюция, экология, анатомия
2. Для молекулярных вопросов - вспомни структуры ДНК/РНК, белки, ферменты
3. Для клеточной биологии - рассмотри органеллы и их функции
4. Для генетики - примени законы Менделя, вспомни о ДНК, РНК, мутациях
5. Для эволюции - рассмотри естественный отбор, адаптации
6. Проверь биологическую логику ответа

В конце дай ТОЛЬКО цифру правильного варианта (0, 1, 2 или 3).
""",

    "math": """
Ты математик. Реши математическую задачу, используя строгое логическое рассуждение:

1. Определи тип задачи: алгебра, геометрия, тригонометрия, математический анализ, теория вероятностей
2. Выпиши все данные и неизвестные
3. Вспомни соответствующие теоремы, формулы, свойства
4. Реши задачу шаг за шагом, показывая логические переходы
5. Для геометрии - сделай мысленный чертеж или представь пространственные отношения
6. Проверь решение на наличие ошибок в вычислениях

После решения напиши ТОЛЬКО номер правильного ответа (0-3). Без пояснений.
""",

    "economics": """
Ты экономист-аналитик. Проанализируй экономический вопрос с помощью системного подхода:

1. Определи раздел экономики: микроэкономика, макроэкономика, международная экономика
2. Выдели ключевые экономические концепции (спрос/предложение, издержки, ВВП, инфляция и т.д.)
3. Примени соответствующие экономические модели и теории
4. Проанализируй причинно-следственные связи
5. Рассмотри альтернативные варианты и их последствия
6. Проверь экономическую логику ответа

После анализа выбери и укажи ТОЛЬКО номер верного варианта (0, 1, 2 или 3).
""",

    "psychology": """
Ты профессиональный психолог. Проанализируй психологический вопрос, используя научный подход:

1. Определи область психологии: когнитивная, социальная, клиническая, развивающая, биологическая
2. Вспомни соответствующие психологические теории и исследования
3. Примени принципы научной методологии в психологии
4. Рассмотри различные перспективы и подходы
5. Отдели научно обоснованные факты от популярных представлений
6. Проверь соответствие современным психологическим знаниям

В результате дай ТОЛЬКО цифру правильного ответа (0-3).
""",

    "health": """
Ты специалист в области здравоохранения. Ответь на вопрос о здоровье, используя медицинский подход:

1. Определи тему: анатомия, физиология, питание, общественное здоровье, медицинские процедуры
2. Опирайся на научно доказанные медицинские факты
3. Рассмотри анатомические структуры и физиологические процессы
4. Для вопросов о питании - вспомни питательные вещества и их функции
5. Для медицинских тем - примени принципы доказательной медицины
6. Проверь соответствие современным медицинским знаниям

После анализа напиши ТОЛЬКО номер верного варианта (0, 1, 2 или 3).
""",

    "history": """
Ты историк-аналитик. Проанализируй исторический вопрос, используя критическое мышление:

1. Определи исторический период и регион
2. Вспомни ключевые события, личности, даты
3. Рассмотри причинно-следственные связи исторических процессов
4. Учти различные исторические источники и перспективы
5. Отдели исторические факты от интерпретаций
6. Проверь хронологическую последовательность

В конце укажи ТОЛЬКО цифру правильного ответа (0, 1, 2 или 3).
""",

    "law": """
Ты юрист-аналитик. Проанализируй правовой вопрос, используя юридическое мышление:

1. Определи область права: конституционное, гражданское, уголовное, административное
2. Выдели юридические понятия и принципы
3. Примени соответствующие правовые нормы и прецеденты
4. Проанализируй факты и их юридическую квалификацию
5. Рассмотри все стороны правовой ситуации
6. Проверь логическую последовательность юридических аргументов

После анализа дай ТОЛЬКО номер правильного варианта (0-3).
""",

    "engineering": """
Ты инженер-специалист. Реши инженерную задачу, используя системный подход:

1. Определи инженерную дисциплину: механика, электротехника, строительство и т.д.
2. Выдели технические требования и ограничения
3. Вспомни соответствующие инженерные принципы и формулы
4. Примени метод пошагового проектирования и анализа
5. Проверь техническую осуществимость и безопасность
6. Учти практические аспекты реализации

После решения напиши ТОЛЬКО номер верного ответа (0, 1, 2 или 3).
""",

    "computer science": """
Ты специалист по компьютерным наукам. Реши задачу по информатике, используя алгоритмическое мышление:

1. Определи тему: алгоритмы, структуры данных, программирование, компьютерные сети, базы данных
2. Проанализируй проблему и выдели ключевые элементы
3. Вспомни соответствующие алгоритмы, структуры, принципы
4. Примени логическое рассуждение для решения
5. Для алгоритмических вопросов - пошагово проанализируй выполнение
6. Проверь корректность логических выводов

В результате дай ТОЛЬКО цифру правильного варианта (0-3).
""",

    "business": """
Ты бизнес-эксперт. Проанализируй бизнес-вопрос, используя стратегическое мышление:

1. Определи бизнес-концепцию: менеджмент, маркетинг, финансы, стратегия
2. Выдели ключевые бизнес-принципы и модели
3. Проанализируй ситуацию с точки зрения эффективности и результативности
4. Рассмотри финансовые, маркетинговые и управленческие аспекты
5. Оцени риски и возможности
6. Проверь бизнес-логику предлагаемых решений

После анализа укажи ТОЛЬКО номер правильного ответа (0, 1, 2 или 3).
""",

    "philosophy": """
Ты философ-аналитик. Проанализируй философский вопрос, используя критическое мышление:

1. Определи философскую область: эпистемология, этика, метафизика, логика, философия науки
2. Выдели ключевые философские понятия и теории
3. Примени методы философского анализа и аргументации
4. Рассмотри различные философские позиции и их обоснования
5. Проанализируй логическую структуру аргументов
6. Проверь непротиворечивость и обоснованность выводов

В конце напиши ТОЛЬКО цифру правильного варианта (0, 1, 2 или 3).
""",

    "other": """
Ты эрудит с широкими знаниями. Ответь на вопрос, используя междисциплинарный подход:

1. Проанализируй вопрос и определи его общую тематику
2. Вспомни соответствующие знания из разных областей
3. Примени логическое и критическое мышление
4. Рассмотри вопрос с разных перспектив
5. Проверь факты и логику
6. Исключи маловероятные или неподтвержденные варианты

После анализа дай ТОЛЬКО номер правильного ответа (0, 1, 2 или 3).
"""
}


FEW_SHOT_PROMPTS = {
    "history": """Вот примеры формата ответов:

Пример 1:
Вопрос: В каком году произошла Куликовская битва?
Варианты ответа:
0. 1223 год
1. 1242 год  
2. 1380 год
3. 1480 год
Ответ: 2

Пример 2:
Вопрос: Кто был первым императором России?
Варианты ответа:
0. Иван Грозный
1. Петр I
2. Екатерина II
3. Николай I
Ответ: 1""",

    "math": """Вот примеры формата ответов:

Пример 1:
Вопрос: Чему равно значение выражения 2x + 3y при x=2, y=4?
Варианты ответа:
0. 10
1. 12
2. 14
3. 16
Ответ: 3

Пример 2:
Вопрос: Какая производная у функции f(x) = x²?
Варианты ответа:
0. x
1. 2x
2. 2
3. 0
Ответ: 1""",

    "physics": """Вот примеры формата ответов:

Пример 1:
Вопрос: С какой силой притягивается тело массой 1 кг к Земле?
Варианты ответа:
0. 1 Н
1. 9.8 Н
2. 10 Н
3. 100 Н
Ответ: 1

Пример 2:
Вопрос: Какой закон описывает сохранение энергии?
Варианты ответа:
0. Закон Ома
1. Закон Ньютона
2. Закон сохранения энергии
3. Закон Архимеда
Ответ: 2""",

    "psychology": """Вот примеры формата ответов:

Пример 1:
Вопрос: Кто основал психоанализ?
Варианты ответа:
0. Карл Юнг
1. Зигмунд Фрейд
2. Иван Павлов
3. Абрахам Маслоу
Ответ: 1

Пример 2:
Вопрос: Что такое когнитивный диссонанс?
Варианты ответа:
0. Конфликт между мыслями и действиями
1. Нарушение памяти
2. Депрессивное состояние
3. Тревожное расстройство
Ответ: 0""",

    "law": """Вот примеры формата ответов:

Пример 1:
Вопрос: Какой кодекс регулирует гражданские отношения в России?
Варианты ответа:
0. Уголовный кодекс
1. Гражданский кодекс
2. Трудовой кодекс
3. Налоговый кодекс
Ответ: 1

Пример 2:
Вопрос: Что такое презумпция невиновности?
Варианты ответа:
0. Обвиняемый должен доказывать свою невиновность
1. Обвиняемый считается невиновным, пока вина не доказана
2. Судья всегда прав
3. Закон обратной силы не имеет
Ответ: 1""",

    "business": """Вот примеры формата ответов:

Пример 1:
Вопрос: Что такое SWOT-анализ?
Варианты ответа:
0. Анализ сильных и слабых сторон, возможностей и угроз
1. Анализ финансовых показателей
2. Анализ рынка конкурентов
3. Анализ кадрового состава
Ответ: 0

Пример 2:
Вопрос: Что означает ROI?
Варианты ответа:
0. Return On Investment - возврат на инвестиции
1. Rate Of Interest - процентная ставка
2. Risk Of Inflation - риск инфляции
3. Revenue Over Income - доход поверх дохода
Ответ: 0""",

    "biology": """Вот примеры формата ответов:

Пример 1:
Вопрос: Какой процесс называют фотосинтезом?
Варианты ответа:
0. Дыхание растений
1. Поглощение воды корнями
2. Преобразование света в химическую энергию
3. Размножение растений
Ответ: 2

Пример 2:
Вопрос: Кто открыл структуру ДНК?
Варианты ответа:
0. Мендель
1. Павлов
2. Уотсон и Крик
3. Дарвин
Ответ: 2""",

    "engineering": """Вот примеры формата ответов:

Пример 1:
Вопрос: Что измеряется в Омах?
Варианты ответа:
0. Напряжение
1. Сопротивление
2. Ток
3. Мощность
Ответ: 1

Пример 2:
Вопрос: Какой закон описывает зависимость тока от напряжения?
Варианты ответа:
0. Закон Ома
1. Закон Ньютона
2. Закон Архимеда
3. Закон Паскаля
Ответ: 0""",

    "economics": """Вот примеры формата ответов:

Пример 1:
Вопрос: Что такое инфляция?
Варианты ответа:
0. Рост цен и снижение покупательной способности
1. Снижение цен
2. Стабильность цен
3. Рост производства
Ответ: 0

Пример 2:
Вопрос: Что измеряет ВВП?
Варианты ответа:
0. Уровень безработицы
1. Объем производства товаров и услуг
2. Уровень инфляции
3. Доходы населения
Ответ: 1""",

    "chemistry": """Вот примеры формата ответов:

Пример 1:
Вопрос: Какая формула воды?
Варианты ответа:
0. H2O
1. CO2
2. NaCl
3. O2
Ответ: 0

Пример 2:
Вопрос: Что такое pH?
Варианты ответа:
0. Показатель кислотности
1. Показатель плотности
2. Показатель температуры
3. Показатель давления
Ответ: 0""",

    "philosophy": """Вот примеры формата ответов:

Пример 1:
Вопрос: Кто автор "Государства"?
Варианты ответа:
0. Аристотель
1. Платон
2. Сократ
3. Декарт
Ответ: 1

Пример 2:
Вопрос: Что такое "cogito ergo sum"?
Варианты ответа:
0. "Мыслю, следовательно существую"
1. "Знание - сила"
2. "Бытие определяет сознание"
3. "Всё течет, всё изменяется"
Ответ: 0""",

    "health": """Вот примеры формата ответов:

Пример 1:
Вопрос: Что измеряет тонометр?
Варианты ответа:
0. Температуру тела
1. Артериальное давление
2. Уровень сахара в крови
3. Частоту пульса
Ответ: 1

Пример 2:
Вопрос: Какой витамин вырабатывается под воздействием солнца?
Варианты ответа:
0. Витамин A
1. Витамин C
2. Витамин D
3. Витамин B12
Ответ: 2""",

    "computer science": """Вот примеры формата ответов:

Пример 1:
Вопрос: Что такое алгоритм?
Варианты ответа:
0. Устройство компьютера
1. Программа на Python
2. Последовательность действий для решения задачи
3. База данных
Ответ: 2

Пример 2:
Вопрос: Какой язык программирования создал Гвидо ван Россум?
Варианты ответа:
0. Java
1. Python
2. C++
3. JavaScript
Ответ: 1""",

    "other": """Вот примеры формата ответов:

Пример 1:
Вопрос: Какая самая длинная река в мире?
Варианты ответа:
0. Амазонка
1. Нил
2. Янцзы
3. Миссисипи
Ответ: 1

Пример 2:
Вопрос: Кто написал "Войну и мир"?
Варианты ответа:
0. Достоевский
1. Толстой
2. Чехов
3. Тургенев
Ответ: 1"""
}





PHYSICS_PROMPT = """Ты - физик-преподаватель. Решай задачи систематически:

УНИВЕРСАЛЬНЫЙ АЛГОРИТМ РЕШЕНИЯ ФИЗИЧЕСКИХ ЗАДАЧ:

1. 🔍 АНАЛИЗ УСЛОВИЯ:
   - Выдели все числовые данные с единицами измерения
   - Определи физическую величину, которую нужно найти
   - Определи раздел физики: механика, оптика, термодинамика, электродинамика, квантовая физика

2. 📚 ВЫБОР ФИЗИЧЕСКОЙ МОДЕЛИ:
   - Определи, какие физические законы применимы:
     * Механика: Ньютона, сохранения энергии/импульса, кинематика
     * Оптика: геометрическая (линзы, зеркала) или волновая (интерференция, дифракция)
     * Электродинамика: Кулона, Ома, Фарадея, Максвелла
     * Термодинамика: законы термодинамики, уравнение состояния
     * Атомная физика: модели атома, спектры, энергетические уровни

3. 📐 МАТЕМАТИЧЕСКАЯ ФОРМУЛИРОВКА:
   - Запиши соответствующие уравнения
   - Проверь размерности всех величин
   - Сделай необходимые преобразования единиц

4. 🧮 ПОШАГОВОЕ РЕШЕНИЕ:
   - Решай последовательно, показывая каждый шаг
   - Проверяй промежуточные результаты на здравый смысл
   - Учитывай граничные условия и начальные данные

5. ✅ ВАЛИДАЦИЯ ОТВЕТА:
   - Проверь размерность полученного результата
   - Оцени правдоподобность численного значения
   - Сравни с вариантами ответов

КЛЮЧЕВЫЕ ПРИНЦИПЫ ДЛЯ РАЗНЫХ РАЗДЕЛОВ:

А) МЕХАНИКА:
   - Сохранение энергии: E_кин + E_пот = const
   - Сохранение импульса: Σp_i = const
   - Уравнения движения: x = x₀ + v₀t + at²/2

Б) ОПТИКА:
   - Формула линзы: 1/F = 1/f₁ + 1/f₂ (знаки по правилу знаков)
   - Волновая оптика: λ = v/f, интерференционные условия

В) ЭЛЕКТРОДИНАМИКА:
   - Закон Кулона: F = k·q₁q₂/r²
   - Напряженность поля: E = F/q

Г) ВОЛНЫ И КОЛЕБАНИЯ:
   - Уравнение колебаний: x = A·sin(ωt + φ)
   - Связь параметров: v = λ·f, ω = 2πf

ВАЖНО: Не пропускай проверку размерностей! Это часто помогает найти ошибку.

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...)."""




HEALTH_PROMPT = """Ты - опытный врач-клиницист. Используй системный подход:

ОБЩАЯ СТРАТЕГИЯ МЕДИЦИНСКОГО АНАЛИЗА:

1. 🩺 СБОР ИНФОРМАЦИИ:
   - Выдели ключевую жалобу пациента
   - Проанализируй анамнез: возраст, хронические заболевания, операции
   - Оцени жизненные показатели: температура, пульс, давление, дыхание

2. 🧠 ДИФФЕРЕНЦИАЛЬНАЯ ДИАГНОСТИКА:
   - Составь список возможных состояний (от наиболее к наименее вероятным)
   - Используй принцип "Common things are common" (частое встречается чаще)
   - Не забывай об опасных для жизни состояниях

3. 📊 ОЦЕНКА ФИЗИКАЛЬНОГО ОСМОТРА:
   - Систематически оценивай по системам: сердечно-сосудистая, дыхательная, неврологическая и т.д.
   - Обращай внимание на ключевые признаки: цвет, температура, пульсация, отёк

4. 🏥 ВЫБОР ДИАГНОСТИКИ И ЛЕЧЕНИЯ:
   - Принцип "от простого к сложному" в диагностике
   - Приоритет срочных и опасных для жизни состояний
   - Учитывай риски и пользу вмешательств

5. 📋 СПЕЦИФИЧНЫЕ ПОДХОДЫ:
   
   А) ХИРУРГИЧЕСКИЕ СЛУЧАИ:
      - Проверяй послеоперационные осложнения: кровотечение, инфекция, тромбоз
      - Оценивай кровоснабжение: цвет, температура, пульсация
      - При потере пульсации → думай о сосудистых проблемах
   
   Б) АНАТОМИЯ:
      - Восстанавливай трёхмерное представление анатомических структур
      - Помни типичные расположения и взаимоотношения органов
   
   В) БИОХИМИЯ И ФИЗИОЛОГИЯ:
      - Понимай метаболические пути и энергетический баланс
      - Знай нормальные физиологические показатели
   
   Г) ЭПИДЕМИОЛОГИЯ:
      - Различай эффекты возраста, периода и когорты
      - Анализируй популяционные закономерности

ОБЩИЙ ПРИНЦИП: В медицине часто выбирается НАИБОЛЕЕ БЕЗОПАСНЫЙ вариант при неопределённости.

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...)."""




LAW_PROMPT = """Ты - юрист-аналитик. Используй системный правовой анализ:

ОБЩАЯ МЕТОДОЛОГИЯ ПРАВОВОГО АНАЛИЗА:

1. ⚖️ ИДЕНТИФИКАЦИЯ ПРАВОВОЙ ОБЛАСТИ:
   - Определи отрасль права: уголовное, гражданское, административное, конституционное
   - Определи юрисдикцию: национальное/международное право, конкретная страна/штат

2. 🔍 АНАЛИЗ ФАКТИЧЕСКИХ ОБСТОЯТЕЛЬСТВ:
   - Выдели все существенные факты дела
   - Определи правовые отношения между сторонами
   - Идентифицируй ключевые события в хронологическом порядке

3. 📚 ПРИМЕНЕНИЕ ПРАВОВЫХ ПРИНЦИПОВ:
   
   А) УГОЛОВНОЕ ПРАВО (общие принципы):
      - Actus reus: наличие противоправного деяния
      - Mens rea: наличие вины (умысел, неосторожность, небрежность)
      - Causation: причинно-следственная связь
      - Concurrence: совпадение умысла и действия
      - Защиты: необходимая оборона, крайняя необходимость
   
   Б) ГРАЖДАНСКОЕ ПРОЦЕССУАЛЬНОЕ ПРАВО:
      - Подсудность (jurisdiction): предметная и территориальная
      - Право на иск (standing)
      - Процессуальные сроки и формальности
   
   В) ДОГОВОРНОЕ ПРАВО:
      - Элементы договора: предложение, принятие, встречное удовлетворение
      - Нарушение договора и средства защиты

4. 🧠 ЛОГИЧЕСКОЕ РАССУЖДЕНИЕ:
   - Применяй дедукцию от общих принципов к конкретному случаю
   - Рассматривай альтернативные правовые позиции
   - Оценивай силу аргументов каждой стороны

5. 📝 ФОРМУЛИРОВКА ВЫВОДА:
   - Делай вывод на основе применения права к фактам
   - Указывай наиболее вероятный исход с учётом прецедентов
   - Будь готов обосновать свой вывод

ВАЖНО: В праве часто правильный ответ - не "что хотелось бы", а "что следует из применения права".

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...)."""




PHILOSOPHY_PROMPT = """Ты - философ-исследователь. Используй методологический анализ:

ОБЩАЯ МЕТОДОЛОГИЯ ФИЛОСОФСКОГО АНАЛИЗА:

1. 🧭 ИДЕНТИФИКАЦИЯ ФИЛОСОФСКОЙ ПРОБЛЕМАТИКИ:
   - Определи центральную философскую проблему: метафизика, эпистемология, этика, философия сознания и т.д.
   - Выдели ключевые понятия и их определения

2. 📚 КОНТЕКСТУАЛИЗАЦИЯ:
   - Определи философскую традицию/школу: античная, средневековая, современная
   - Идентифицируй возможного философа по стилю и содержанию
   - Учитывай историко-философский контекст

3. 🔍 АНАЛИЗ КЛЮЧЕВЫХ КОНЦЕПЦИЙ:
   
   А) МЕТАФИЗИКА (природа реальности):
      - Проблема существования: что существует и в каком смысле?
      - Проблема свободы воли: детерминизм vs индетерминизм
      - Проблема сознания и тела
   
   Б) ЭПИСТЕМОЛОГИЯ (теория познания):
      - Проблема источников знания: эмпиризм vs рационализм
      - Проблема обоснования знания
      - Проблема скептицизма
   
   В) ЭТИКА (моральная философия):
      - Нормативная этика: деонтология, утилитаризм, этика добродетели
      - Метаэтика: природа моральных суждений
      - Прикладная этика
   
   Г) ФИЛОСОФИЯ ЯЗЫКА:
      - Проблема значения и референции
      - Отношение языка к реальности

4. 🧩 СИСТЕМАТИЧЕСКОЕ РАССУЖДЕНИЕ:
   - Анализируй логическую структуру утверждений
   - Выявляй предпосылки и импликации
   - Проверяй на непротиворечивость
   - Рассматривай контрпримеры и возражения

5. 📝 ФОРМУЛИРОВКА ФИЛОСОФСКОЙ ПОЗИЦИИ:
   - Точно воспроизводи ключевые тезисы философов
   - Избегай упрощений и вульгаризаций
   - Учитывай нюансы философской аргументации

ОБЩИЙ ПРИНЦИП: В философии часто правильные ответы кажутся парадоксальными и требуют глубокого понимания концепций.

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...)."""

In [ ]:
PROMPTS = {
"physics": """
Ты - опытный физик-преподаватель. Решай задачи систематически:

УНИВЕРСАЛЬНЫЙ АЛГОРИТМ РЕШЕНИЯ ФИЗИЧЕСКИХ ЗАДАЧ:

1.  АНАЛИЗ УСЛОВИЯ:
   - Выдели все числовые данные с единицами измерения
   - Определи физическую величину, которую нужно найти
   - Определи раздел физики: механика, оптика, термодинамика, электродинамика, квантовая физика

2.  ВЫБОР ФИЗИЧЕСКОЙ МОДЕЛИ:
   - Определи, какие физические законы применимы:
     * Механика: Ньютона, сохранения энергии/импульса, кинематика
     * Оптика: геометрическая (линзы, зеркала) или волновая (интерференция, дифракция)
     * Электродинамика: Кулона, Ома, Фарадея, Максвелла
     * Термодинамика: законы термодинамики, уравнение состояния
     * Атомная физика: модели атома, спектры, энергетические уровни

3.  МАТЕМАТИЧЕСКАЯ ФОРМУЛИРОВКА:
   - Запиши соответствующие уравнения
   - Проверь размерности всех величин
   - Сделай необходимые преобразования единиц

4.  ПОШАГОВОЕ РЕШЕНИЕ:
   - Решай последовательно, показывая каждый шаг
   - Проверяй промежуточные результаты на здравый смысл
   - Учитывай граничные условия и начальные данные

5.  ВАЛИДАЦИЯ ОТВЕТА:
   - Проверь размерность полученного результата
   - Оцени правдоподобность численного значения
   - Сравни с вариантами ответов

КЛЮЧЕВЫЕ ПРИНЦИПЫ ДЛЯ РАЗНЫХ РАЗДЕЛОВ:

А) МЕХАНИКА:
   - Сохранение энергии: E_кин + E_пот = const
   - Сохранение импульса: Σp_i = const
   - Уравнения движения: x = x₀ + v₀t + at²/2

Б) ОПТИКА:
   - Формула линзы: 1/F = 1/f₁ + 1/f₂ (знаки по правилу знаков)
   - Волновая оптика: λ = v/f, интерференционные условия

В) ЭЛЕКТРОДИНАМИКА:
   - Закон Кулона: F = k·q₁q₂/r²
   - Напряженность поля: E = F/q

Г) ВОЛНЫ И КОЛЕБАНИЯ:
   - Уравнение колебаний: x = A·sin(ωt + φ)
   - Связь параметров: v = λ·f, ω = 2πf

ВАЖНО: Если нужной формулы тут нет, то вспомни ее сам, ты все помнишь, ты же профи
ВАЖНО: Не пропускай проверку размерностей! Это часто помогает найти ошибку.

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...).
""",

"biology": """
Ты - ведущий биолог-исследователь с системным мышлением. Используй иерархический подход к биологическому анализу:

МЕТОДОЛОГИЯ БИОЛОГИЧЕСКОГО АНАЛИЗА:

1. ИДЕНТИФИКАЦИЯ УРОВНЯ БИОЛОГИЧЕСКОЙ ОРГАНИЗАЦИИ:
   - Определи релевантный уровень анализа: молекулярный, клеточный, организменный, популяционный, экосистемный.
   - Установи связи между уровнями (как молекулярные процессы влияют на организменные функции).

2. МОЛЕКУЛЯРНЫЙ И БИОХИМИЧЕСКИЙ АНАЛИЗ (при необходимости):
   - Воспроизведи ключевые биохимические пути: метаболизм, сигнальные каскады, экспрессия генов.
   - Вспомни структуры и функции: ДНК/РНК (транскрипция, трансляция), белки (ферменты, рецепторы, структура-функция), липиды, углеводы.
   - Примени принципы молекулярной биологии: центральная догма, регуляция генов, посттрансляционные модификации.

3. КЛЕТОЧНЫЙ И ФИЗИОЛОГИЧЕСКИЙ АНАЛИЗ:
   - Рассмотри функции органелл в контексте целой клетки.
   - Проанализируйте клеточные процессы: деление (митоз/мейоз), апоптоз, транспорт, коммуникация.
   - Для физиологических вопросов: гомеостаз, регуляторные системы (нервная, эндокринная), принцип обратной связи.

4. ГЕНЕТИЧЕСКИЙ И ЭВОЛЮЦИОННЫЙ АНАЛИЗ:
   - Примени принципы наследственности: законы Менделя, сцепленное наследование, эпистаз.
   - Рассмотри механизмы изменчивости: мутации, рекомбинация, горизонтальный перенос генов.
   - Используй эволюционные принципы: естественный отбор, дрейф генов, адаптация, филогенетические отношения.
   - Помни о различных типах отбора: стабилизирующий, направленный, дизруптивный.

5. ЭКОЛОГИЧЕСКИЙ И ЭКОСИСТЕМНЫЙ ПОДХОД:
   - Проанализируй взаимодействия: хищник-жертва, симбиоз, конкуренция.
   - Рассмотри поток энергии и круговорот веществ в экосистемах.
   - Учти популяционную динамику: рост, ограничивающие факторы, структура популяций.

6. ИНТЕГРАЦИЯ И ПРОВЕРКА НА СОГЛАСОВАННОСТЬ:
   - Проверь, что ответ согласуется со всеми уровнями биологической организации.
   - Убедись в отсутствии противоречий с фундаментальными биологическими принципами.
   - Рассмотри исключения и пограничные случаи в биологии.

ОБЩИЙ ПРИНЦИП: Биологические системы иерархичны и эмерджентны — свойства целого не сводятся к свойствам частей. Правильный ответ должен учитывать эту сложность.

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...).
""",

    "math": """
Ты - математик с фундаментальным образованием. Решай задачу, применяя строгую математическую дедукцию:

МЕТОДОЛОГИЯ МАТЕМАТИЧЕСКОГО РАССУЖДЕНИЯ:

1. ТОЧНАЯ ФОРМАЛИЗАЦИЯ ПРОБЛЕМЫ:
   - Определи математическую область: алгебра, геометрия, математический анализ, теория вероятностей, дискретная математика, теория чисел.
   - Переформулируй задачу на формальном языке, явно определи все переменные, множества, функции и отношения.
   - Определи тип требуемого ответа: доказательство, вычисление, построение, классификация.

2. ВЫБОР И ОБОСНОВАНИЕ МАТЕМАТИЧЕСКОГО АППАРАТА:
   - Подбери соответствующие теоремы, леммы, формулы и методы решения.
   - Для доказательств: определи стратегию (прямое доказательство, от противного, математическая индукция).
   - Для вычислений: выбери оптимальный алгоритм или метод (аналитический, численный, комбинаторный).

3. СТРОГАЯ ПОСЛЕДОВАТЕЛЬНОСТЬ ВЫВОДА:
   - Каждый шаг должен быть логически обоснован предыдущим или известной теоремой.
   - Для геометрических задач: строй четкое мысленное представление, используй известные свойства фигур, теоремы планиметрии/стереометрии.
   - Для алгебраических задач: выполняй преобразования, сохраняя эквивалентность.

4. АНАЛИЗ ОСОБЫХ СЛУЧАЕВ И ОГРАНИЧЕНИЙ:
   - Проверь область определения функций и допустимые значения переменных.
   - Рассмотри вырожденные и пограничные случаи.
   - Для вероятностных задач: четко определи пространство элементарных событий.

5. ВЕРИФИКАЦИЯ РЕШЕНИЯ:
   - Проверь соответствие ответа исходным условиям.
   - Выполни обратную подстановку или альтернативный метод для подтверждения.
   - Убедись в полноте решения (учтены все случаи, нет потери решений).
   - Проверь на наличие скрытых допущений.

6. ОЦЕНКА КРАСОТЫ И ЭФФЕКТИВНОСТИ РЕШЕНИЯ:
   - Идеальное решение должно быть не только правильным, но и элегантным.
   - Стремись к максимальной простоте и ясности рассуждений.

ОБЩИЙ ПРИНЦИП: Математика — это наука о строгих доказательствах. Правильное решение должно быть не просто интуитивно верным, но и логически безупречным, с явно указанными основаниями для каждого шага.

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...).
""",

    "economics": """
Ты - экономист-теоретик с опытом аналитической работы. Проведи экономический анализ, используя формальные модели:

МЕТОДОЛОГИЯ ЭКОНОМИЧЕСКОГО АНАЛИЗА:

1. ФРЕЙМИРОВАНИЕ ЭКОНОМИЧЕСКОЙ ПРОБЛЕМЫ:
   - Определи область: микроэкономика (поведение агентов, рынки), макроэкономика (экономика в целом), экономика развития, экономика труда, международная экономика.
   - Четко сформулируй экономический вопрос и гипотезы.

2. ВЫБОР И ПРИМЕНЕНИЕ ЭКОНОМИЧЕСКИХ МОДЕЛЕЙ:
   - Для микроэкономики: используй модели спроса и предложения, теорию потребителя (кривые безразличия, бюджетные ограничения), теорию фирмы (издержки, максимизация прибыли), рыночные структуры (совершенная конкуренция, монополия, олигополия).
   - Для макроэкономики: примени модели совокупного спроса/предложения, модели экономического роста (Солоу, эндогенного роста), модели открытой экономики (Манделла-Флеминга).
   - Используй соответствующие графические и математические представления.

3. АНАЛИЗ РАВНОВЕСИЯ И ДИНАМИКИ:
   - Определи состояние равновесия и условия его достижения.
   - Проанализируй, как система реагирует на экзогенные шоки (изменения предпочтений, технологий, политики).
   - Рассмотри краткосрочные и долгосрочные последствия.

4. ОЦЕНКА ЭФФЕКТИВНОСТИ И РАСПРЕДЕЛЕНИЯ:
   - Примени критерии эффективности по Парето.
   - Проанализируй компромиссы (trade-offs): например, между инфляцией и безработицей (кривая Филлипса), между эффективностью и равенством.
   - Рассмотри проблемы "провалов рынка" и роль государства.

5. ЭМПИРИЧЕСКАЯ И ИНСТИТУЦИОНАЛЬНАЯ ВАЛИДАЦИЯ:
   - Сопоставь теоретические предсказания с эмпирическими фактами и историческими примерами.
   - Учти институциональный контекст: права собственности, контракты, нормативные ограничения.
   - Рассмотри поведенческие аспекты (ограниченная рациональность, прокрастинация).

6. ФОРМУЛИРОВКА ПОЛИТИЧЕСКИХ ВЫВОДОВ И ОГРАНИЧЕНИЙ:
   - Четко сформулируй экономически обоснованные рекомендации.
   - Укажи допущения модели и границы применимости выводов.
   - Оцени риски и непредвиденные последствия политических вмешательств.

ОБЩИЙ ПРИНЦИП: Экономика изучает выбор в условиях ограниченных ресурсов. Правильный ответ должен отражать понимание стимулов, компромиссов и распределительных последствий, а также учитывать как теоретические модели, так и реальные институты.

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...).
""",

    "psychology": """
Ты - научный психолог с междисциплинарным подходом. Проведи психологический анализ, основываясь на доказательных исследованиях:

МЕТОДОЛОГИЯ НАУЧНО-ПСИХОЛОГИЧЕСКОГО АНАЛИЗА:

1. ИДЕНТИФИКАЦИЯ ПСИХОЛОГИЧЕСКОЙ ПЕРСПЕКТИВЫ:
   - Определи основной подход: биопсихологический/нейронаучный, когнитивный, поведенческий, психоаналитический/психодинамический, гуманистический, социально-культурный, эволюционный.
   - Установи связь с соответствующими уровнями анализа: биологический, индивидуальный, социальный.

2. БИОЛОГИЧЕСКИЕ И НЕЙРОНАУЧНЫЕ ОСНОВЫ (при необходимости):
   - Рассмотри релевантные структуры мозга и нейронные системы.
   - Вспомни роль нейротрансмиттеров, гормонов, генетических факторов.
   - Примени принципы психофизиологии и поведенческой генетики.

3. КОГНИТИВНЫЙ И ПОВЕДЕНЧЕСКИЙ АНАЛИЗ:
   - Используй модели обработки информации: восприятие, внимание, память, мышление, принятие решений.
   - Примени принципы обучения: классическое и оперантное обусловливание, социальное научение.
   - Рассмотри когнитивные искажения и эвристики (работы Канемана и Тверски).

4. СОЦИАЛЬНЫЕ И РАЗВИВАЮЩИЕ АСПЕКТЫ:
   - Проанализируй социальное влияние: конформизм, послушание, групповая динамика, атрибуция.
   - Учти возрастные изменения и кризисы развития (теории Пиаже, Эриксона, Выготского).
   - Рассмотри культурные различия в психологических процессах.

5. КЛИНИЧЕСКИЕ И ИНДИВИДУАЛЬНЫЕ РАЗЛИЧИЯ:
   - Различай нормальные вариации и патологические состояния.
   - Примени диагностические критерии (DSM-5, МКБ-11) там, где это уместно.
   - Рассмотри индивидуальные различия: личность (пятифакторная модель), интеллект, мотивация.

6. КРИТИЧЕСКАЯ ОЦЕНКА И ИНТЕГРАЦИЯ:
   - Оцени методологическую строгость соответствующих исследований (экспериментальные vs корреляционные исследования, размер выборки, контроль переменных).
   - Отдели научно обоснованные факты от популярных мифов и упрощений.
   - Интегрируй данные из разных психологических направлений для целостного понимания.
   - Укажи степень уверенности в выводах, основанную на качестве доказательств.

ОБЩИЙ ПРИНЦИП: Психология — это наука о поведении и психических процессах. Правильный ответ должен основываться на эмпирических данных, учитывать множественность причин поведения и избегать редукционизма к单一 объяснению.

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...).
""",
    "health": """
Ты - опытный врач-клиницист. Используй системный подход:

ОБЩАЯ СТРАТЕГИЯ МЕДИЦИНСКОГО АНАЛИЗА:

1.  СБОР ИНФОРМАЦИИ:
   - Выдели ключевую жалобу пациента
   - Проанализируй анамнез: возраст, хронические заболевания, операции
   - Оцени жизненные показатели: температура, пульс, давление, дыхание

2.  ДИФФЕРЕНЦИАЛЬНАЯ ДИАГНОСТИКА:
   - Составь список возможных состояний (от наиболее к наименее вероятным)
   - Используй принцип "Common things are common" (частое встречается чаще)
   - Не забывай об опасных для жизни состояниях

3.  ОЦЕНКА ФИЗИКАЛЬНОГО ОСМОТРА:
   - Систематически оценивай по системам: сердечно-сосудистая, дыхательная, неврологическая и т.д.
   - Обращай внимание на ключевые признаки: цвет, температура, пульсация, отёк

4.  ВЫБОР ДИАГНОСТИКИ И ЛЕЧЕНИЯ:
   - Принцип "от простого к сложному" в диагностике
   - Приоритет срочных и опасных для жизни состояний
   - Учитывай риски и пользу вмешательств

5.  СПЕЦИФИЧНЫЕ ПОДХОДЫ:
   
   А) ХИРУРГИЧЕСКИЕ СЛУЧАИ:
      - Проверяй послеоперационные осложнения: кровотечение, инфекция, тромбоз
      - Оценивай кровоснабжение: цвет, температура, пульсация
      - При потере пульсации → думай о сосудистых проблемах
   
   Б) АНАТОМИЯ:
      - Восстанавливай трёхмерное представление анатомических структур
      - Помни типичные расположения и взаимоотношения органов
   
   В) БИОХИМИЯ И ФИЗИОЛОГИЯ:
      - Понимай метаболические пути и энергетический баланс
      - Знай нормальные физиологические показатели
   
   Г) ЭПИДЕМИОЛОГИЯ:
      - Различай эффекты возраста, периода и когорты
      - Анализируй популяционные закономерности

ОБЩИЙ ПРИНЦИП: В медицине часто выбирается НАИБОЛЕЕ БЕЗОПАСНЫЙ вариант при неопределённости.

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...).
""",

    "history": """
Ты историк-аналитик. Проанализируй исторический вопрос, используя критическое мышление:

1. Определи исторический период и регион
2. Вспомни ключевые события, личности, даты
3. Рассмотри причинно-следственные связи исторических процессов
4. Учти различные исторические источники и перспективы
5. Отдели исторические факты от интерпретаций
6. Проверь хронологическую последовательность

В конце укажи ТОЛЬКО цифру правильного ответа (0, 1, 2 или 3).
""",

    "law": """
Ты - опытный юрист-аналитик. Используй системный правовой анализ:

ОБЩАЯ МЕТОДОЛОГИЯ ПРАВОВОГО АНАЛИЗА:

1.  ИДЕНТИФИКАЦИЯ ПРАВОВОЙ ОБЛАСТИ:
   - Определи отрасль права: уголовное, гражданское, административное, конституционное
   - Определи юрисдикцию: национальное/международное право, конкретная страна/штат

2.  АНАЛИЗ ФАКТИЧЕСКИХ ОБСТОЯТЕЛЬСТВ:
   - Выдели все существенные факты дела
   - Определи правовые отношения между сторонами
   - Идентифицируй ключевые события в хронологическом порядке

3.  ПРИМЕНЕНИЕ ПРАВОВЫХ ПРИНЦИПОВ:
   
   А) УГОЛОВНОЕ ПРАВО (общие принципы):
      - Actus reus: наличие противоправного деяния
      - Mens rea: наличие вины (умысел, неосторожность, небрежность)
      - Causation: причинно-следственная связь
      - Concurrence: совпадение умысла и действия
      - Защиты: необходимая оборона, крайняя необходимость
   
   Б) ГРАЖДАНСКОЕ ПРОЦЕССУАЛЬНОЕ ПРАВО:
      - Подсудность (jurisdiction): предметная и территориальная
      - Право на иск (standing)
      - Процессуальные сроки и формальности
   
   В) ДОГОВОРНОЕ ПРАВО:
      - Элементы договора: предложение, принятие, встречное удовлетворение
      - Нарушение договора и средства защиты

4.  ЛОГИЧЕСКОЕ РАССУЖДЕНИЕ:
   - Применяй дедукцию от общих принципов к конкретному случаю
   - Рассматривай альтернативные правовые позиции
   - Оценивай силу аргументов каждой стороны

5.  ФОРМУЛИРОВКА ВЫВОДА:
   - Делай вывод на основе применения права к фактам
   - Указывай наиболее вероятный исход с учётом прецедентов
   - Будь готов обосновать свой вывод

ВАЖНО: В праве часто правильный ответ - не "что хотелось бы", а "что следует из применения права".

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...)
""",

"engineering": """
Ты - инженер-проектировщик с системным мышлением. Используй строгий инженерный подход:

ИНЖЕНЕРНАЯ МЕТОДОЛОГИЯ АНАЛИЗА И РЕШЕНИЯ:

1. КЛАССИФИКАЦИЯ ЗАДАЧИ И НОРМАТИВНАЯ БАЗА:
   - Определи инженерную дисциплину: механика, электротехника, строительство, термодинамика, гидравлика, химическая технология и т.д.
   - Идентифицируй применимые нормативные документы (ГОСТы, СНиПы, ISO, отраслевые стандарты).

2. СБОР И АНАЛИЗ ИСХОДНЫХ ДАННЫХ:
   - Выдели все заданные параметры, условия функционирования и технические требования (ТЗ).
   - Чётко определи граничные условия и ограничения: физические, экономические, временные, экологические.
   - Проанализируй риски: технические отказы, безопасность, влияние на окружающую среду.

3. ПРИМЕНЕНИЕ ФУНДАМЕНТАЛЬНЫХ ПРИНЦИПОВ:
   - Используй соответствующие фундаментальные законы: сохранения (энергии, массы, импульса), термодинамики, механики сплошных сред, теории цепей и др.
   - Примени ключевые инженерные формулы и соотношения.
   - Учти специфические эффекты и допущения для данной дисциплины (например, ламинарное/турбулентное течение, линейность/нелинейность материалов).

4. СИСТЕМНОЕ ПРОЕКТИРОВАНИЕ И РАСЧЁТ:
   - Разбей систему на логические компоненты или подсистемы.
   - Проведи анализ "сверху-вниз" (от требований к деталям) и "снизу-вверх" (проверка совместимости компонентов).
   - Для каждого этапа выполни необходимые расчёты, моделирование или оценку.

5. ВЕРИФИКАЦИЯ И ВАЛИДАЦИЯ РЕШЕНИЯ:
   - Проверь соответствие решения исходным требованиям и ограничениям.
   - Оцени технико-экономическую эффективность, надёжность и живучесть системы.
   - Проведи мысленный анализ на предмет слабых мест, "узких горлышек" и возможных режимов отказа.
   - Убедись в соблюдении всех норм безопасности и эргономики.

6. ОФОРМЛЕНИЕ ИНЖЕНЕРНОГО ВЫВОДА:
   - Решение должно быть технически обоснованным, воспроизводимым и безопасным.
   - В условиях неопределённости выбирай консервативный подход с запасом прочности/производительности.

ОБЩИЙ ПРИНЦИП: Хорошее инженерное решение — это оптимальный компромисс между функциональностью, надёжностью, стоимостью и безопасностью.

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...).
""",

    "computer science": """
Ты - ведущий инженер-программист (Staff/Principal Engineer). Решай задачу, применяя фундаментальные принципы компьютерных наук:

МЕТОДОЛОГИЯ АНАЛИЗА ВЫЧИСЛИТЕЛЬНЫХ СИСТЕМ:

1. АБСТРАКЦИЯ И КЛАССИФИКАЦИЯ ПРОБЛЕМЫ:
   - Определи домен задачи: алгоритмы, структуры данных, архитектура, сети, ОС, базы данных, теория вычислений.
   - Сформулирую проблему на формальном или псевдокодовом языке, отделив суть от деталей.
   - Определи класс сложности проблемы (если применимо): P, NP, константное/логарифмическое/линейное время и т.д.

2. ВЫБОР И АНАЛИЗ ИНСТРУМЕНТОВ:
   - Подбери подходящие абстрактные структуры данных (деревья, хеш-таблицы, графы, очереди) и алгоритмические парадигмы (жадные, "разделяй и властвуй", ДП, поиск с возвратом).
   - Вспомни ключевые теоремы, принципы и паттерны (например, CAP-теорема, принцип локальности, шаблоны проектирования).

3. СИСТЕМНОЕ МЫШЛЕНИЕ И ОПТИМИЗАЦИЯ:
   - Проанализируй trade-offs: время vs память, согласованность vs доступность, читаемость vs производительность.
   - Рассмотри проблему на разных уровнях абстракции: от математической модели до аппаратной реализации.
   - Определи "узкие места" и критические пути выполнения.

4. КОРРЕКТНОСТЬ И ВЕРИФИКАЦИЯ:
   - Докажи или обоснуй корректность подхода (инварианты, условия завершения).
   - Проанализируй краевые случаи (edge cases) и условия гонки (race conditions).
   - Оцени вычислительную сложность (Big O) в худшем, среднем и лучшем случае.

5. ПРАКТИЧЕСКИЕ АСПЕКТЫ РЕАЛИЗАЦИИ:
   - Учти особенности языков программирования, ОС или сред исполнения, если они заданы контекстом.
   - Рассмотри вопросы масштабируемости, параллелизма и распределённости системы.
   - Подумай об устойчивости к ошибкам и отказоустойчивости.

ОБЩИЙ ПРИНЦИП: Фундаментальные структуры и алгоритмы важнее синтаксиса конкретного языка. Стремись к оптимальному балансу между элегантностью, эффективностью и практичностью.

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...).
""",

    "business": """
Ты - стратегический консультант топ-уровня (как из McKinsey, BCG). Анализируй бизнес-кейс, применяя структурированные фреймворки:

МЕТОДОЛОГИЯ СТРАТЕГИЧЕСКОГО БИЗНЕС-АНАЛИЗА:

1. ФРЕЙМИРОВАНИЕ ПРОБЛЕМЫ И ЦЕЛЕЙ:
   - Определи ключевую бизнес-проблему: рост, прибыльность, операционная эффективность, выход на рынок, M&A и т.д.
   - Чётко сформулирую гипотезу и конечную цель анализа.
   - Определи стейкхолдеров и их интересы.

2. СБОР И АНАЛИЗ ДАННЫХ (ФАКТОРНЫЙ АНАЛИЗ):
   - Используй фреймворки для анализа среды: PESTEL (макросреда), Porter's 5 Forces (отрасль), SWOT (позиция компании).
   - Проведи финансовый анализ: рентабельность, ликвидность, структура затрат, инвестиционная привлекательность.
   - Проанализируй рынок: размер, рост, сегментация, доля, поведение клиентов.

3. РАЗРАБОТКА И ОЦЕНКА АЛЬТЕРНАТИВ:
   - Сгенерируй варианты решений, используя матрицы типа BCG, Ansoff или ценностные цепочки.
   - Оцени каждую альтернативу по ключевым критериям: финансовый эффект (NPV, ROI), риски, реализуемость, соответствие стратегии.
   - Проведи количественную оценку, где это возможно.

4. СИНТЕЗ И ПРИНЯТИЕ РЕКОМЕНДАЦИЙ:
   - Сформулируй ясные, конкретные и выполнимые рекомендации.
   - Определи ключевые метрики успеха (OKR/KPI) и план реализации (дорожную карту).
   - Предусмотри план управления рисками и сценарии на случай непредвиденных обстоятельств.

5. КОММУНИКАЦИЯ И ВНЕДРЕНИЕ:
   - Рекомендации должны быть убедительными, логичными и подкреплёнными данными.
   - Учти организационные и культурные барьеры на пути внедрения.

ОБЩИЙ ПРИНЦИП: В бизнесе нет единственно верного ответа, но есть наиболее обоснованное решение, основанное на данных, логике и четком понимании компромиссов.

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...).
""",

    "philosophy": """
Ты - философ-исследователь. Используй методологический анализ:

ОБЩАЯ МЕТОДОЛОГИЯ ФИЛОСОФСКОГО АНАЛИЗА:

1. ИДЕНТИФИКАЦИЯ ФИЛОСОФСКОЙ ПРОБЛЕМАТИКИ:
   - Определи центральную философскую проблему: метафизика, эпистемология, этика, философия сознания, политическая философия, эстетика.
   - Выдели ключевые понятия, дихотомии и их точные определения.

2.  КОНТЕКСТУАЛИЗАЦИЯ И АТРИБУЦИЯ:
   - Определи философскую традицию/школу: античная, средневековая, рационализм, эмпиризм, немецкая классика, аналитическая, континентальная.
   - Идентифицируй возможного философа по стилю аргументации, терминологии и историческому периоду.
   - Установи диалог между различными философскими позициями по данному вопросу.

3. СТРУКТУРНЫЙ АНАЛИЗ КЛЮЧЕВЫХ КОНЦЕПЦИЙ:
   
   А) МЕТАФИЗИКА И ОНТОЛОГИЯ:
      - Проблема существования: реализм vs номинализм, субстанция vs свойства.
      - Причинность, детерминизм, свобода воли и проблема времени.
      - Проблема сознания и тела: дуализм, физикализм, функционализм.
   
   Б) ЭПИСТЕМОЛОГИЯ:
      - Источники и границы знания: эмпиризм, рационализм, априорное знание.
      - Проблема обоснования: фундаментализм, когерентизм, инфинитизм.
      - Ответ на скептицизм.
   
   В) ЭТИКА И МОРАЛЬНАЯ ПСИХОЛОГИЯ:
      - Нормативные теории: деонтология (Кант), утилитаризм (Милль, Бентам), этика добродетели (Аристотель).
      - Метаэтика: когнитивизм/нонкогнитивизм, моральный реализм/антиреализм.
      - Структура морального мотивации и действия.
   
   Г) ЛОГИКА И ФИЛОСОФИЯ ЯЗЫКА:
      - Анализ логической формы утверждений и аргументов.
      - Теории значения: референция, истинность, интенциональность.
      - Речевые акты и прагматика.

4. СИСТЕМАТИЧЕСКОЕ РАССУЖДЕНИЕ И КРИТИКА:
   - Реконструируй аргумент в стандартной форме (посылки -> заключение).
   - Выяви имплицитные предпосылки, скрытые допущения и логические следствия.
   - Проверь внутреннюю непротиворечивость и внешнюю совместимость с другими убеждениями.
   - Сформулируй возможные контрпримеры, дилеммы и возражения.
   - Рассмотри силу и слабость аргументации с разных философских позиций.

5. ИНТЕРПРЕТАЦИЯ И СИНТЕЗ:
   - Точно воспроизводи тонкие нюансы философских позиций, избегая карикатур.
   - Различай буквальное и метафорическое прочтение текстов.
   - Сформулируй сбалансированный вывод, отражающий сложность и глубину проблемы.

ОБЩИЙ ПРИНЦИП: В философии правильный ответ часто заключается не в простом выборе, а в понимании структуры аргументации, силы конкурирующих позиций и умозрительных следствий каждой из них.

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...).
""",

    "other": """
Ты - ведущий учёный-эрудит и методолог (по типу Аристотеля или Дени Дидро). Отвечай на вопрос, применяя универсальный структурированный подход к знанию:

УНИВЕРСАЛЬНАЯ МЕТОДОЛОГИЯ АНАЛИЗА:

1. КАТЕГОРИЗАЦИЯ И СИСТЕМАТИЗАЦИЯ ВОПРОСА:
   - Определи фундаментальную область знания: естественные науки (физика, химия, биология), гуманитарные (история, лингвистика), социальные (экономика, социология) или междисциплинарная проблема.
   - Разложи сложный вопрос на элементарные составляющие или подвопросы.

2. ПРИМЕНЕНИЕ СООТВЕТСТВУЮЩЕЙ ЭПИСТЕМОЛОГИИ:
   - Для фактологических вопросов: используй принципы научного метода (верифицируемость, фальсифицируемость, воспроизводимость).
   - Для исторических/интерпретационных вопросов: применяй принципы герменевтики и критики источников.
   - Для логических/математических вопросов: строй дедуктивные рассуждения.

3. МЕЖДИСЦИПЛИНАРНЫЙ СИНТЕЗ И ПРОВЕРКА НА СОГЛАСОВАННОСТЬ:
   - Рассмотри проблему с позиций нескольких смежных дисциплин.
   - Проверь, не противоречат ли выводы установленным фактам из других областей (например, биологическое утверждение не должно противоречить законам химии).
   - Выяви парадоксы или пробелы в объяснении.

4. КРИТИЧЕСКАЯ ОЦЕНКА ДОСТОВЕРНОСТИ И ЛОГИКИ:
   - Отдели установленные факты от гипотез, мнений или псевдонаучных утверждений.
   - Проанализируй причинно-следственные связи, корреляцию и каузальность.
   - Оцени силу и качество доказательств для каждого возможного варианта ответа.

5. ФОРМУЛИРОВКА РАЗВЁРНУТОГО И ОБОСНОВАННОГО ВЫВОДА:
   - Если вопрос допускает однозначный ответ, найди его через исключение и проверку.
   - Если вопрос сложный или дискуссионный, определи наиболее обоснованную позицию, взвесив все "за" и "против".
   - Избегай логических ошибок (софизмов) и когнитивных искажений.

ОБЩИЙ ПРИНЦИП: Истинное знание системно и непротиворечиво. Наиболее вероятный ответ — тот, который лучше всего согласуется с максимально широким массивом проверенных фактов и логических принципов, выдерживая критическую проверку с разных сторон.

Ответ давай ТОЛЬКО индексом (0, 1, 2, ...).
"""
}


FEW_SHOT_PROMPTS = {
    "history": """Вот примеры формата ответов:

Пример 1:
Вопрос: В каком году произошла Куликовская битва?
Варианты ответа:
0. 1223 год
1. 1242 год  
2. 1380 год
3. 1480 год
Ответ: 2

Пример 2:
Вопрос: Кто был первым императором России?
Варианты ответа:
0. Иван Грозный
1. Петр I
2. Екатерина II
3. Николай I
Ответ: 1""",

    "math": """Вот примеры формата ответов:

Пример 1:
Вопрос: Чему равно значение выражения 2x + 3y при x=2, y=4?
Варианты ответа:
0. 10
1. 12
2. 14
3. 16
Ответ: 3

Пример 2:
Вопрос: Какая производная у функции f(x) = x²?
Варианты ответа:
0. x
1. 2x
2. 2
3. 0
Ответ: 1""",

    "physics": """Вот примеры формата ответов для физических задач:

Пример 1:
Вопрос: Тело массой 5 кг поднимают на высоту 2 м. Какую работу совершают против силы тяжести? (g=10 м/с²)
Варианты ответа:
0. 10 Дж
1. 50 Дж
2. 100 Дж
3. 200 Дж
Ответ: 2

Пример 2:
Вопрос: Линза с оптической силой 5 диоптрий. Каково её фокусное расстояние?
Варианты ответа:
0. 0.2 м
1. 0.5 м
2. 2 м
3. 5 м
Ответ: 0""",

    "psychology": """Вот примеры формата ответов:

Пример 1:
Вопрос: Кто основал психоанализ?
Варианты ответа:
0. Карл Юнг
1. Зигмунд Фрейд
2. Иван Павлов
3. Абрахам Маслоу
Ответ: 1

Пример 2:
Вопрос: Что такое когнитивный диссонанс?
Варианты ответа:
0. Конфликт между мыслями и действиями
1. Нарушение памяти
2. Депрессивное состояние
3. Тревожное расстройство
Ответ: 0""",

"law": """Вот примеры формата ответов для юридических вопросов:

Пример 1:
Вопрос: Какой вид юридической ответственности наступает за нарушение договора?
Варианты ответа:
0. Уголовная
1. Административная
2. Гражданская
3. Дисциплинарная
Ответ: 2

Пример 2:
Вопрос: С какого возраста наступает полная дееспособность по общему правилу?
Варианты ответа:
0. 14 лет
1. 16 лет
2. 18 лет
3. 21 год
Ответ: 2""",

    "business": """Вот примеры формата ответов:

Пример 1:
Вопрос: Что такое SWOT-анализ?
Варианты ответа:
0. Анализ сильных и слабых сторон, возможностей и угроз
1. Анализ финансовых показателей
2. Анализ рынка конкурентов
3. Анализ кадрового состава
Ответ: 0

Пример 2:
Вопрос: Что означает ROI?
Варианты ответа:
0. Return On Investment - возврат на инвестиции
1. Rate Of Interest - процентная ставка
2. Risk Of Inflation - риск инфляции
3. Revenue Over Income - доход поверх дохода
Ответ: 0""",

    "biology": """Вот примеры формата ответов:

Пример 1:
Вопрос: Какой процесс называют фотосинтезом?
Варианты ответа:
0. Дыхание растений
1. Поглощение воды корнями
2. Преобразование света в химическую энергию
3. Размножение растений
Ответ: 2

Пример 2:
Вопрос: Кто открыл структуру ДНК?
Варианты ответа:
0. Мендель
1. Павлов
2. Уотсон и Крик
3. Дарвин
Ответ: 2""",

    "engineering": """Вот примеры формата ответов:

Пример 1:
Вопрос: Что измеряется в Омах?
Варианты ответа:
0. Напряжение
1. Сопротивление
2. Ток
3. Мощность
Ответ: 1

Пример 2:
Вопрос: Какой закон описывает зависимость тока от напряжения?
Варианты ответа:
0. Закон Ома
1. Закон Ньютона
2. Закон Архимеда
3. Закон Паскаля
Ответ: 0""",

    "economics": """Вот примеры формата ответов:

Пример 1:
Вопрос: Что такое инфляция?
Варианты ответа:
0. Рост цен и снижение покупательной способности
1. Снижение цен
2. Стабильность цен
3. Рост производства
Ответ: 0

Пример 2:
Вопрос: Что измеряет ВВП?
Варианты ответа:
0. Уровень безработицы
1. Объем производства товаров и услуг
2. Уровень инфляции
3. Доходы населения
Ответ: 1""",

    "chemistry": """Вот примеры формата ответов:

Пример 1:
Вопрос: Какая формула воды?
Варианты ответа:
0. H2O
1. CO2
2. NaCl
3. O2
Ответ: 0

Пример 2:
Вопрос: Что такое pH?
Варианты ответа:
0. Показатель кислотности
1. Показатель плотности
2. Показатель температуры
3. Показатель давления
Ответ: 0""",

"philosophy": """Вот примеры формата ответов для философских вопросов:

Пример 1:
Вопрос: Какой философ считается основоположником диалектики?
Варианты ответа:
0. Сократ
1. Платон
2. Аристотель
3. Гегель
Ответ: 3

Пример 2:
Вопрос: Кто автор работы "Так говорил Заратустра"?
Варианты ответа:
0. Кант
1. Ницше
2. Шопенгауэр
3. Кьеркегор
Ответ: 1""",

"health": """Вот примеры формата ответов для медицинских вопросов:

Пример 1:
Вопрос: Какой витамин необходим для нормального свёртывания крови?
Варианты ответа:
0. Витамин A
1. Витамин C
2. Витамин D
3. Витамин K
Ответ: 3

Пример 2:
Вопрос: Нормальная частота сердечных сокращений у взрослого человека в покое?
Варианты ответа:
0. 40-60 уд/мин
1. 60-100 уд/мин
2. 100-120 уд/мин
3. 120-140 уд/мин
Ответ: 1""",

    "computer science": """Вот примеры формата ответов:

Пример 1:
Вопрос: Что такое алгоритм?
Варианты ответа:
0. Устройство компьютера
1. Программа на Python
2. Последовательность действий для решения задачи
3. База данных
Ответ: 2

Пример 2:
Вопрос: Какой язык программирования создал Гвидо ван Россум?
Варианты ответа:
0. Java
1. Python
2. C++
3. JavaScript
Ответ: 1""",

    "other": """Вот примеры формата ответов:

Пример 1:
Вопрос: Какая самая длинная река в мире?
Варианты ответа:
0. Амазонка
1. Нил
2. Янцзы
3. Миссисипи
Ответ: 1

Пример 2:
Вопрос: Кто написал "Войну и мир"?
Варианты ответа:
0. Достоевский
1. Толстой
2. Чехов
3. Тургенев
Ответ: 1"""
}


In [19]:
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import re
import os
import gc
import json
from pathlib import Path
from typing import Optional, Dict, Any
import time
from tqdm import tqdm
from typing import Literal, Optional, Tuple, List

/Users/karlkorhonen/проекты/Data_analysis/LLM/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class LLM:
    def __init__(
        self,
        model_name="Qwen/Qwen2.5-14B-Instruct",
        device="cuda",
        _prompts=PROMPTS,
        _few_shot_prompts=FEW_SHOT_PROMPTS,
        model=None,
        tokenizer=None,
        quantization_config=None,
        debug=False,
        deep_debug=False,
        use_llm_parsing=True,
        use_selfcheck=False,
        llm_cot_generation=True,
        llm_few_shot_generation=True
    ) -> None:
        self.DEBUG = debug
        self.DEEP_DEBUG = deep_debug
        self.debug_logs = []
        self.USE_LLM_PARSING = use_llm_parsing
        self.USE_SELFCHECK = use_selfcheck
        self.LLM_COT_GENERATION = llm_cot_generation
        self.LLM_FEW_SHOT_GENERATION = llm_few_shot_generation
        try:
            self.prompts = _prompts
            self.few_shot_prompts = _few_shot_prompts
            self.model_name = model_name
            self.device = device if torch.cuda.is_available() and device == "cuda" else "cpu"
            print(f"🏋️‍♂️ Модель: {self.model_name}")
            print(f"🖥 Устройство: {self.device}")
            print(f"⚙️  CoT генерация: {'ВКЛ' if llm_cot_generation else 'ВЫКЛ'}")
            print(f"⚙️  Few-shot генерация: {'ВКЛ' if llm_few_shot_generation else 'ВЫКЛ'}")
            if model is not None and tokenizer is not None:
                print("✅ Используем переданные модель и токенайзер")
                self.model = model
                self.tokenizer = tokenizer
                return
            print(f"📥 Загрузка {self.model_name}...")
            self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            if quantization_config is None:
                quantization_config = BitsAndBytesConfig(
                    bnb_4bit_quant_type="nf4",
                    bnb_4bit_compute_dtype=torch.float16,
                    bnb_4bit_use_double_quant=True,
                    load_in_4bit=True,
                )
            self.model = AutoModelForCausalLM.from_pretrained(
                model_name,
                quantization_config=quantization_config,
                device_map="auto",
                trust_remote_code=True
            )
            print(f"✅ Модель {model_name} успешно загружена!")

        except Exception as e:
            print(f"❌ Ошибка загрузки: {e}")
            print("Пробуем загрузить без квантования...")
            try:
                self.tokenizer = AutoTokenizer.from_pretrained(
                    model_name,
                    trust_remote_code=True
                )
                if self.tokenizer.pad_token is None:
                    self.tokenizer.pad_token = self.tokenizer.eos_token

                self.model = AutoModelForCausalLM.from_pretrained(
                    model_name,
                    device_map="auto",
                    torch_dtype=torch.float16,
                    trust_remote_code=True
                )
                print("✅ Модель загружена без квантования")
            except Exception as e2:
                print(f"❌ Критическая ошибка: {e2}")
                raise e2

    def get_answer_method(self, use_selfcheck=None):
        if use_selfcheck is None:
            use_selfcheck = self.USE_SELFCHECK
        return self.generate_answer_selfcheck if use_selfcheck else self.generate_answer

    def generate_answer(
        self,
        question:str,
        encoded_options,
        category:str,
        dramatic:bool = True,
        tokens:int = 1000,
        temperature:float = 0.1,
        few_shot = True,
        use_llm_parsing=None,
        use_selfcheck=None,
        llm_cot_generation=None,
        llm_few_shot_generation=None,
        force_diversity: bool = False
    )->int:

        if use_llm_parsing is None:
            use_llm_parsing = self.USE_LLM_PARSING
        if use_selfcheck is None:
            use_selfcheck = self.USE_SELFCHECK
        if llm_cot_generation is None:
            llm_cot_generation = self.LLM_COT_GENERATION
        if llm_few_shot_generation is None:
            llm_few_shot_generation = self.LLM_FEW_SHOT_GENERATION

        if use_selfcheck:
            return self.generate_answer_selfcheck(
                question=question,
                encoded_options=encoded_options,
                category=category,
                dramatic=dramatic,
                tokens=tokens,
                temperature=temperature,
                use_llm_parsing=use_llm_parsing,
                llm_cot_generation=llm_cot_generation,
                llm_few_shot_generation=llm_few_shot_generation
            )

        self._log("generate_answer", "начало", {
            "category": category,
            "question_len": len(question),
            "options_raw_preview": str(encoded_options)[:200],
            "temperature": temperature,
            "use_llm_parsing": use_llm_parsing,
            "use_selfcheck": use_selfcheck,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation,
            "force_diversity": force_diversity
        }, "DEBUG")

        options = self._options_parser(encoded_options)
        self._log("generate_answer", "распарсенные опции", {
            "count": len(options),
            "first_3": options[:3] if len(options) > 3 else options
        }, "DEBUG")

        if len(options) <= 1 and options[0] == "Варианты не предоставлены":
            self._log("generate_answer", "ОШИБКА: нет вариантов для вопроса", {
                "question": question[:200]
            }, "DEBUG")
            return 0

        result = self.generate_prompt(
            question=question,
            encoded_options=options,
            topic=category,
            drammatic=dramatic,
            few_shot=few_shot,
            llm_cot_generation=llm_cot_generation,
            llm_few_shot_generation=llm_few_shot_generation
        )

        if result is None:
            self._log("generate_answer", "промпт не сгенерирован", None, "DEBUG")
            return 0
        system_prompt, user_prompt = result

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]

        text = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = self.tokenizer(text, return_tensors="pt").to(self.device)

        generation_kwargs = {
            "input_ids": inputs.input_ids,
            "attention_mask": inputs.attention_mask,
            "max_new_tokens": tokens,
            "pad_token_id": self.tokenizer.pad_token_id,
        }

        if force_diversity:
            generation_kwargs["temperature"] = temperature
            generation_kwargs["do_sample"] = True
            generation_kwargs["top_p"] = 0.9
        elif temperature > 0:
            generation_kwargs["temperature"] = temperature
            generation_kwargs["do_sample"] = True

            if category in ['history', 'philosophy', 'law', 'psychology', 'other']:
                generation_kwargs["top_p"] = 0.9
            elif category in ['math', 'physics', 'engineering', 'computer science', 'chemistry']:
                if temperature < 0.3:
                    generation_kwargs["do_sample"] = False
        else:

            generation_kwargs["temperature"] = 0.1
            generation_kwargs["do_sample"] = False

        self._log("generate_answer", "параметры генерации", {
            "temperature": generation_kwargs.get("temperature", 0),
            "do_sample": generation_kwargs.get("do_sample", False),
            "top_p": generation_kwargs.get("top_p", None),
            "category": category,
            "force_diversity": force_diversity
        }, "DEBUG")

        with torch.no_grad():
            generated_ids = self.model.generate(**generation_kwargs)

        response = self.tokenizer.decode(
            generated_ids[0][inputs.input_ids.shape[1]:],
            skip_special_tokens=True
        )

        parsed = self.parse_answer_index(response, use_llm_parsing=use_llm_parsing)

        self._log_response(
            "MAIN_MODEL_RESPONSE",
            response,
            parsed,
            None,
            {
                "category": category,
                "question": question[:100],
                "use_llm_parsing": use_llm_parsing,
                "use_selfcheck": use_selfcheck,
                "llm_cot_generation": llm_cot_generation,
                "llm_few_shot_generation": llm_few_shot_generation,
                "temperature": generation_kwargs.get("temperature", 0),
                "do_sample": generation_kwargs.get("do_sample", False),
                "response_length": len(response)
            }
        )

        self._log("generate_answer", "завершено", {
            "parsed": parsed,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation,
            "generation_params": {
                "temp": generation_kwargs.get("temperature", 0),
                "do_sample": generation_kwargs.get("do_sample", False)
            }
        }, "DEBUG")

        return parsed

    def generate_answer_selfcheck(
        self,
        question:str,
        encoded_options,
        category:str,
        dramatic:bool = True,
        tokens:int = 1000,
        temperature:float = 0.1,
        use_llm_parsing=None,
        use_selfcheck=None,
        llm_cot_generation=None,
        llm_few_shot_generation=None
    )->int:

        if use_llm_parsing is None:
            use_llm_parsing = self.USE_LLM_PARSING
        if use_selfcheck is None:
            use_selfcheck = self.USE_SELFCHECK
        if llm_cot_generation is None:
            llm_cot_generation = self.LLM_COT_GENERATION
        if llm_few_shot_generation is None:
            llm_few_shot_generation = self.LLM_FEW_SHOT_GENERATION

        self._log("generate_answer_selfcheck", "начало", {
            "category": category,
            "use_llm_parsing": use_llm_parsing,
            "use_selfcheck": use_selfcheck,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEBUG")

        main_response = self.generate_answer(
            question=question,
            encoded_options=encoded_options,
            category=category,
            dramatic=dramatic,
            tokens=tokens,
            temperature=temperature,
            use_llm_parsing=use_llm_parsing,
            use_selfcheck=False,
            llm_cot_generation=llm_cot_generation,
            llm_few_shot_generation=llm_few_shot_generation
        )

        selfcheck = self._generate_selfcheck_prompt(
            question=question,
            topic=category,
            encoded_options=encoded_options,
            predicted_answer=main_response,
            drammatic=dramatic,
            use_llm_parsing=use_llm_parsing,
            llm_cot_generation=llm_cot_generation,
            llm_few_shot_generation=llm_few_shot_generation
        )

        if selfcheck is None:
            self._log("generate_answer_selfcheck", "selfcheck промпт не сгенерирован", None, "DEBUG")
            return main_response

        system_prompt, user_prompt = selfcheck

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]

        text = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = self.tokenizer(text, return_tensors="pt").to(self.device)

        with torch.no_grad():
            generated_ids = self.model.generate(
                inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=tokens,
                temperature=temperature,
                do_sample=False,
                pad_token_id=self.tokenizer.pad_token_id,
            )

        response = self.tokenizer.decode(
            generated_ids[0][inputs.input_ids.shape[1]:],
            skip_special_tokens=True
        )

        verified = self.parse_answer_index(response, use_llm_parsing=use_llm_parsing)

        self._log_response(
            "SELFCHECK_RESPONSE",
            response,
            verified,
            None,
            {
                "category": category,
                "main_response": main_response,
                "verified": verified,
                "changed": main_response != verified,
                "use_llm_parsing": use_llm_parsing,
                "use_selfcheck": use_selfcheck,
                "llm_cot_generation": llm_cot_generation,
                "llm_few_shot_generation": llm_few_shot_generation
            }
        )

        self._log("generate_answer_selfcheck", "завершено", {
            "main": main_response,
            "verified": verified,
            "changed": main_response != verified,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEBUG")

        return verified

    def ensemble_vote(
        self,
        question,
        encoded_options,
        category,
        n_votes=3,
        temperatures=[0.1, 0.5, 0.9],
        use_llm_parsing=None,
        use_selfcheck=None,
        llm_cot_generation=None,
        llm_few_shot_generation=None,
        force_diversity = True
    ):
        if use_llm_parsing is None:
            use_llm_parsing = self.USE_LLM_PARSING
        if use_selfcheck is None:
            use_selfcheck = self.USE_SELFCHECK
        if llm_cot_generation is None:
            llm_cot_generation = self.LLM_COT_GENERATION
        if llm_few_shot_generation is None:
            llm_few_shot_generation = self.LLM_FEW_SHOT_GENERATION

        self._log("ensemble_vote", "начало", {
            "category": category,
            "n_votes": n_votes,
            "temperatures": temperatures[:n_votes],
            "use_llm_parsing": use_llm_parsing,
            "use_selfcheck": use_selfcheck,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEBUG")

        votes = []
        for i, temp in enumerate(temperatures[:n_votes]):
            self._log("ensemble_vote", f"голос {i+1}", {
                "temperature": temp,
                "llm_cot_generation": llm_cot_generation,
                "llm_few_shot_generation": llm_few_shot_generation
            }, "DEEP_DEBUG")

            answer = self.generate_answer(
                question=question,
                encoded_options=encoded_options,
                category=category,
                temperature=temp,
                tokens=1000,
                use_llm_parsing=use_llm_parsing,
                use_selfcheck=use_selfcheck,
                llm_cot_generation=llm_cot_generation,
                llm_few_shot_generation=llm_few_shot_generation,
                force_diversity=force_diversity
            )
            parsed = self.parse_answer_index(str(answer))
            votes.append(parsed)

            self._log("ensemble_vote", f"голос {i+1} результат", {
                "answer": answer,
                "parsed": parsed,
                "temperature": temp,
                "llm_cot_generation": llm_cot_generation,
                "llm_few_shot_generation": llm_few_shot_generation
            }, "DEEP_DEBUG")

        from collections import Counter
        vote_counts = Counter(votes)
        most_common = vote_counts.most_common(1)[0]

        self._log("ensemble_vote", "результат", {
            "votes": votes,
            "distribution": dict(vote_counts),
            "final": most_common[0],
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEBUG")

        return most_common[0], dict(vote_counts)

    def ask_with_verification(
        self,
        question,
        encoded_options,
        category,
        temperature=0.1,
        use_llm_parsing=None,
        use_selfcheck=None,
        llm_cot_generation=None,
        llm_few_shot_generation=None
    ):
        if use_llm_parsing is None:
            use_llm_parsing = self.USE_LLM_PARSING
        if use_selfcheck is None:
            use_selfcheck = self.USE_SELFCHECK
        if llm_cot_generation is None:
            llm_cot_generation = self.LLM_COT_GENERATION
        if llm_few_shot_generation is None:
            llm_few_shot_generation = self.LLM_FEW_SHOT_GENERATION

        self._log("ask_with_verification", "начало", {
            "category": category,
            "temperature": temperature,
            "use_llm_parsing": use_llm_parsing,
            "use_selfcheck": use_selfcheck,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEBUG")

        first_answer = self.generate_answer(
            question=question,
            encoded_options=encoded_options,
            category=category,
            temperature=temperature,
            tokens=400,
            use_llm_parsing=use_llm_parsing,
            use_selfcheck=use_selfcheck,
            llm_cot_generation=llm_cot_generation,
            llm_few_shot_generation=llm_few_shot_generation
        )

        self._log("ask_with_verification", "первый ответ", {
            "first_answer": first_answer,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEEP_DEBUG")

        res = self._generate_selfcheck_prompt(
            question=question,
            topic=category,
            encoded_options=encoded_options,
            predicted_answer=first_answer,
            drammatic=True,
            use_llm_parsing=use_llm_parsing,
            llm_cot_generation=llm_cot_generation,
            llm_few_shot_generation=llm_few_shot_generation
        )

        if res is None:
            self._log("ask_with_verification", "ошибка генерации selfcheck", None, "DEBUG")
            return 0, 0, 0

        system_prompt, user_prompt = res
        check_response = self.direct_prompt(
            user_prompt=user_prompt,
            system_prompt=system_prompt,
            tokens=300,
            temperature=0.1
        )

        verified_answer = self.parse_answer_index(check_response, use_llm_parsing=use_llm_parsing)

        weight = 1.5 if first_answer == verified_answer else 0.7

        self._log("ask_with_verification", "результат", {
            "first": first_answer,
            "verified": verified_answer,
            "weight": weight,
            "agreement": first_answer == verified_answer,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEBUG")

        return verified_answer, weight, first_answer

    def confidence_ensemble_vote(
        self,
        question,
        encoded_options,
        category,
        n_runs=3,
        temperatures=[0.1, 0.3, 0.5],
        use_llm_parsing=None,
        use_selfcheck=None,
        llm_cot_generation=None,
        llm_few_shot_generation=None
    ):
        if use_llm_parsing is None:
            use_llm_parsing = self.USE_LLM_PARSING
        if use_selfcheck is None:
            use_selfcheck = self.USE_SELFCHECK
        if llm_cot_generation is None:
            llm_cot_generation = self.LLM_COT_GENERATION
        if llm_few_shot_generation is None:
            llm_few_shot_generation = self.LLM_FEW_SHOT_GENERATION

        self._log("confidence_ensemble_vote", "начало", {
            "category": category,
            "n_runs": n_runs,
            "temperatures": temperatures[:n_runs],
            "use_llm_parsing": use_llm_parsing,
            "use_selfcheck": use_selfcheck,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEBUG")

        results = []

        for i, temp in enumerate(temperatures[:n_runs]):
            self._log("confidence_ensemble_vote", f"запуск {i+1}", {
                "temperature": temp,
                "llm_cot_generation": llm_cot_generation,
                "llm_few_shot_generation": llm_few_shot_generation
            }, "DEEP_DEBUG")

            answer, weight, first_answer = self.ask_with_verification(
                question=question,
                encoded_options=encoded_options,
                category=category,
                temperature=temp,
                use_llm_parsing=use_llm_parsing,
                use_selfcheck=use_selfcheck,
                llm_cot_generation=llm_cot_generation,
                llm_few_shot_generation=llm_few_shot_generation
            )

            results.append({
                'answer': answer,
                'weight': weight,
                'first_answer': first_answer,
                'agreement': first_answer == answer
            })

            self._log("confidence_ensemble_vote", f"запуск {i+1} результат", {
                "answer": answer,
                "weight": weight,
                "agreement": first_answer == answer,
                "llm_cot_generation": llm_cot_generation,
                "llm_few_shot_generation": llm_few_shot_generation
            }, "DEEP_DEBUG")

        weighted_counts = {}
        for res in results:
            ans = res['answer']
            weight = res['weight']
            if ans in weighted_counts:
                weighted_counts[ans] += weight
            else:
                weighted_counts[ans] = weight

        best_answer = max(weighted_counts.items(), key=lambda x: x[1])
        total_weight = sum(res['weight'] for res in results)
        final_confidence = best_answer[1] / total_weight if total_weight > 0 else 0

        agreement_rate = sum(1 for res in results if res['agreement']) / len(results) if len(results) > 0 else 0

        self._log("confidence_ensemble_vote", "финальный результат", {
            "best_answer": best_answer[0],
            "confidence": final_confidence,
            "agreement_rate": agreement_rate,
            "weighted_counts": weighted_counts,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEBUG")

        return best_answer[0], weighted_counts, final_confidence, agreement_rate


    def evaluate_dataframe(
        self,
        df,
        question_column="question",
        options_column="options",
        category_column="category",
        answer_column="true_answer",
        method:Literal[
            "ensemble_vote",
            "generate_answer",
            "confidence_ensemble_vote",
            "generate_answer_selfcheck"
        ]="ensemble_vote",
        method_kwargs=None,
        use_llm_parsing=None,
        use_selfcheck=None,
        llm_cot_generation=None,
        llm_few_shot_generation=None
    ):
        if method_kwargs is None:
            method_kwargs = {}

        if use_llm_parsing is None:
            use_llm_parsing = self.USE_LLM_PARSING
        if use_selfcheck is None:
            use_selfcheck = self.USE_SELFCHECK
        if llm_cot_generation is None:
            llm_cot_generation = self.LLM_COT_GENERATION
        if llm_few_shot_generation is None:
            llm_few_shot_generation = self.LLM_FEW_SHOT_GENERATION

        self._log("evaluate_dataframe", "начало", {
            "rows": len(df),
            "method": method,
            "columns": df.columns.tolist(),
            "has_answer_column": answer_column in df.columns,
            "has_actual_answers": answer_column in df.columns and df[answer_column].notna().any()
        }, "DEBUG")

        predictions = []
        processing_times = []
        category_results = {}

        has_actual_answers = False
        if answer_column in df.columns:
            has_actual_answers = df[answer_column].notna().any()
            if has_actual_answers:
                print(f"✅ В данных есть ответы для проверки ({df[answer_column].notna().sum()}/{len(df)})")
            else:
                print("ℹ️  Колонка 'true_answer' есть, но все ответы пустые (только предсказания)")
        pbar = tqdm(total=len(df), desc="Обработка вопросов")

        for idx, row in df.iterrows():
            try:
                start_time = time.time()

                question = str(row[question_column])
                options = row[options_column]
                category = str(row[category_column])

                expected_answer = None
                if answer_column in df.columns and pd.notna(row[answer_column]):
                    expected_answer = self.parse_answer_index(str(row[answer_column]))

                self._log("evaluate_dataframe", f"строка {idx}", {
                    "category": category,
                    "has_expected": expected_answer is not None,
                    "expected": expected_answer,
                    "question_preview": question[:50]
                }, "DEEP_DEBUG")

                method_kwargs_with_defaults = {
                    "use_llm_parsing": use_llm_parsing,
                    "use_selfcheck": use_selfcheck,
                    "llm_cot_generation": llm_cot_generation,
                    "llm_few_shot_generation": llm_few_shot_generation,
                    **method_kwargs
                }

                if method == "generate_answer":
                    predicted = self.generate_answer(
                        question=question,
                        encoded_options=options,
                        category=category,
                        **method_kwargs_with_defaults
                    )
                elif method == "generate_answer_selfcheck":
                    predicted = self.generate_answer_selfcheck(
                        question=question,
                        encoded_options=options,
                        category=category,
                        **method_kwargs_with_defaults
                    )
                elif method == "ensemble_vote":
                    predicted, vote_dist = self.ensemble_vote(
                        question=question,
                        encoded_options=options,
                        category=category,
                        **method_kwargs_with_defaults
                    )

                    if self.DEEP_DEBUG:
                        print(f"ensemble_vote распределение: {vote_dist}")
                elif method == "confidence_ensemble_vote":
                    predicted, weighted_counts, confidence, agreement = self.confidence_ensemble_vote(
                        question=question,
                        encoded_options=options,
                        category=category,
                        **method_kwargs_with_defaults
                    )

                    if self.DEEP_DEBUG:
                        print(f"confidence_ensemble_vote: confidence={confidence}, agreement={agreement}")
                else:
                    raise ValueError(f"Неизвестный метод: {method}")

                predictions.append(predicted)
                processing_time = time.time() - start_time
                processing_times.append(processing_time)

                if category not in category_results:
                    category_results[category] = {
                        'total': 0, 'correct': 0, 'predictions': [], 'truths': []
                    }

                category_results[category]['total'] += 1
                category_results[category]['predictions'].append(predicted)
                if expected_answer is not None:
                    category_results[category]['truths'].append(expected_answer)
                    predicted_parsed = self.parse_answer_index(str(predicted))

                    if predicted_parsed == expected_answer:
                        category_results[category]['correct'] += 1

                        if self.DEEP_DEBUG:
                            print(f"✓ Правильно: предсказано {predicted_parsed}, ожидалось {expected_answer}")
                    else:
                        if self.DEEP_DEBUG:
                            print(f"✗ Ошибка: предсказано {predicted_parsed}, ожидалось {expected_answer}")

                pbar.update(1)
                postfix_info = {
                    'категория': category[:10],
                    'предсказано': self.parse_answer_index(str(predicted))
                }
                if expected_answer is not None:
                    postfix_info['ответ'] = expected_answer
                pbar.set_postfix(postfix_info)

            except Exception as e:
                print(f"\n❌ Ошибка в строке {idx}: {e}")

                self._log("evaluate_dataframe", f"ошибка в строке {idx}", {
                    "error": str(e),
                    "llm_cot_generation": llm_cot_generation,
                    "llm_few_shot_generation": llm_few_shot_generation
                }, "DEBUG")

                predictions.append(0)
                processing_times.append(0)
                pbar.update(1)

        pbar.close()

        self._log("evaluate_dataframe", "завершено", {
            "total_rows": len(df),
            "predictions_made": len(predictions),
            "has_actual_answers": has_actual_answers,
            "method": method
        }, "DEBUG")

        results_df = df.copy()
        results_df['predicted'] = predictions
        results_df['processing_time'] = processing_times
        
        if has_actual_answers:
            try:
                results_df['predicted_parsed'] = results_df['predicted'].apply(
                    lambda x: self.parse_answer_index(str(x)) if pd.notna(x) else 0
                )
                results_df['answer_parsed'] = results_df[answer_column].apply(
                    lambda x: self.parse_answer_index(str(x)) if pd.notna(x) else 0
                )
                results_df['is_correct'] = results_df['predicted_parsed'] == results_df['answer_parsed']

                if self.DEBUG:
                    correct = results_df['is_correct'].sum()
                    total = len(results_df)
                    accuracy = correct / total if total > 0 else 0
                    print(f"📊 Итог точности: {correct}/{total} правильных ({accuracy:.2%})")

            except Exception as e:
                print(f"❌ Ошибка при создании метрик: {e}")
                results_df['is_correct'] = False
        else:
            print("ℹ️  Ответы для проверки отсутствуют, метрики точности не рассчитываются")

        metrics = self._calculate_metrics(
            results_df,
            answer_column if has_actual_answers else None,
            category_results,
            processing_times,
            method
        )

        metrics['use_llm_parsing'] = use_llm_parsing
        metrics['use_selfcheck'] = use_selfcheck
        metrics['llm_cot_generation'] = llm_cot_generation
        metrics['llm_few_shot_generation'] = llm_few_shot_generation
        metrics['has_actual_answers'] = has_actual_answers

        if self.DEBUG and has_actual_answers:
            accuracy = metrics.get('accuracy', 0)
            print(f"📈 Финальная точность: {accuracy:.2%}")

        return results_df, metrics

    def generate_prompt(
        self,
        question,
        topic,
        encoded_options,
        drammatic:bool = True,
        drammatic_prompt = "На тебя возложена огромная надежда и ответственность за мою жизнь и репутацию, не подведи нас",
        few_shot = True,
        llm_cot_generation=True,
        llm_few_shot_generation=True
    ):
        self._log("generate_prompt", "начало", {
            "topic": topic,
            "few_shot": few_shot,
            "encoded_options_raw": str(encoded_options)[:200],
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEEP_DEBUG")

        options = self._options_parser(options=encoded_options)

        self._log("generate_prompt", "распарсенные опции", {
            "count": len(options),
            "options_preview": options[:3] if options else []
        }, "DEBUG")

        if not options or (len(options) == 1 and options[0] == "Варианты не предоставлены"):
            self._log("generate_prompt", "ВНИМАНИЕ: нет вариантов ответа!", {
                "raw_input": str(encoded_options)[:500]
            }, "DEBUG")
            return None

        base_prompt = self.prompts.get(topic,
            "Ты - опытный специалист широкого профиля. Ответь на предоставленный тебе вопрос, выбрав индекс правильного ответа (начиная с нуля)"
            "Дай в ответе только индекс, не давай дополнительных комментариев"
            "Для решения задачи обдумай каждый вариант ответа, подумай, почему он может быть правильным или неправильным"
        )


        cot_section = ""
        if llm_cot_generation:
            cot_instruction = self.generate_cot_instruction(
                question=question,
                topic=topic,
                encoded_options=encoded_options,
                max_tokens=400
            )

            if cot_instruction:
                cot_section = f"""

ИНСТРУКЦИЯ ДЛЯ РЕШЕНИЯ (CHAIN OF THOUGHT):
Реши задачу, следуя этим шагам:
{cot_instruction}

ВАЖНО: После анализа напиши "ОКОНЧАТЕЛЬНЫЙ ОТВЕТ: [индекс]"
"""
            else:
                self._log("generate_prompt", "CoT не сгенерирована, используем fallback", None, "DEBUG")
                cot_section = """

ИНСТРУКЦИЯ ДЛЯ РЕШЕНИЯ:
1. Внимательно прочитай вопрос
2. Проанализируй каждый вариант ответа
3. Исключи неправильные варианты
4. Выбери правильный вариант
5. Напиши только индекс правильного ответа
"""


        few_shot_text = ""
        if few_shot:
            if llm_few_shot_generation:
                few_shot_text = self.generate_contextual_few_shot(
                    question=question,
                    topic=topic,
                    encoded_options=encoded_options,
                    num_examples=2
                )
            else:
                few_shot_text = self.few_shot_prompts.get(topic,
                    """Вот примеры ответов:
Пример 1:
Вопрос: Какая самая длинная река в мире?
Варианты ответа:
0. Амазонка
1. Нил
2. Янцзы
3. Миссисипи
Ответ: 1

Пример 2:
Вопрос: Кто написал "Войну и мир"?
Варианты ответа:
0. Достоевский
1. Толстой
2. Чехов
3. Тургенев
Ответ: 1""" if few_shot else ""
                )

        system_prompt = f"""
{base_prompt}

{f"Примеры решения подобных задач:\n{few_shot_text}\n" if few_shot_text else ""}
{cot_section if llm_cot_generation else ""}

ОБРАТИ ВНИМАНИЕ:
- В этом вопросе {len(options)} вариантов (индексы от 0 до {len(options)-1})
- В ответе дай ТОЛЬКО индекс правильного варианта
- Не пиши объяснений, только число

ФОРМАТ ОТВЕТА: [индекс] (например: 2)
"""

        options_text = "\n".join([f"{ind}. {opt}" for ind, opt in enumerate(options)])

        user_prompt = f"""
Вопрос: {question}

Варианты ответа:
{options_text}

{drammatic_prompt if drammatic else ""}

Ответ (только индекс):"""

        self._log("generate_prompt", "сгенерирован", {
            "options_count": len(options),
            "question_preview": question[:100],
            "first_option": options[0][:100] if options else "нет",
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation,
            "has_cot": bool(cot_section),
            "has_few_shot": bool(few_shot_text)
        }, "DEBUG")

        if self.DEEP_DEBUG:
            print("\n[DEEP_DEBUG] generate_prompt - финальный промпт:")
            print(f"System preview: {system_prompt[:300]}...")
            print(f"\nUser preview: {user_prompt[:300]}...")

        return system_prompt, user_prompt

    def _generate_selfcheck_prompt(
        self,
        question,
        topic,
        encoded_options,
        predicted_answer,
        drammatic:bool = True,
        drammatic_prompt = "На тебя возложена огромная надежда и ответственность за мою жизнь и репутацию, не подведи нас",
        few_shot = True,
        use_llm_parsing=True,
        llm_cot_generation=True,
        llm_few_shot_generation=True
    ):
        self._log("_generate_selfcheck_prompt", "начало", {
            "topic": topic,
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEEP_DEBUG")

        options = self._options_parser(options=encoded_options)
        if not options:
            self._log("_generate_selfcheck_prompt", "ошибка парсинга опций", None, "DEBUG")
            return None

        predicted_idx = self.parse_answer_index(
            str(predicted_answer),
            use_llm_parsing=use_llm_parsing
        )

        option_text = "неизвестный вариант"
        if 0 <= predicted_idx < len(options):
            option_text = options[predicted_idx]

        self._log("_generate_selfcheck_prompt", "индекс предсказания", {
            "raw": str(predicted_answer)[:50],
            "parsed": predicted_idx,
            "option_text": option_text[:50],
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEEP_DEBUG")

        base_prompt = """Ты - эксперт по проверке ответов. Твоя задача - проверить ответ другого эксперта.
Если он правильный - верни тот же индекс. Если неправильный - найди и верни правильный индекс."""

        if llm_cot_generation:
            base_prompt += "\n\nИСПОЛЬЗУЙ ЦЕПОЧКУ РАССУЖДЕНИЙ:\n"
            base_prompt += "1. Проанализируй оригинальный вопрос\n"
            base_prompt += "2. Проверь ответ коллеги на соответствие вопросу\n"
            base_prompt += "3. Если сомневаешься, найди правильный ответ самостоятельно\n"
            base_prompt += "4. Дай окончательный ответ в формате 'ОТВЕТ: [индекс]'"

        if few_shot:
            base_prompt += """
Пример 1:
Вопрос: Какая столица Франции?
Варианты:
0. Лондон
1. Берлин
2. Париж
3. Мадрид
Ответ коллеги: 2
Твой ответ: 2

Пример 2:
Вопрос: 2 + 2 = ?
Варианты:
0. 3
1. 4
2. 5
3. 6
Ответ коллеги: 0
Твой ответ: 1"""

        system_prompt = f"{base_prompt}"

        options_text = "\n".join([f"{i}. {opt}" for i, opt in enumerate(options)])

        user_prompt = f"""
Вопрос: {question}
Варианты ответов:
{options_text}
Ответ коллеги: {predicted_idx} ({option_text})
{drammatic_prompt if drammatic else ""}
Your answer:
"""

        self._log("_generate_selfcheck_prompt", "сгенерирован", {
            "options_count": len(options),
            "llm_cot_generation": llm_cot_generation,
            "llm_few_shot_generation": llm_few_shot_generation
        }, "DEBUG")

        return system_prompt, user_prompt

    def _log(self, method, message, data=None, level="DEBUG"):
        if level == "DEBUG" and not self.DEBUG:
            return
        if level == "DEEP_DEBUG" and not self.DEEP_DEBUG:
            return

        log_entry = {
            "timestamp": time.time(),
            "method": method,
            "message": message,
            "data": data,
            "level": level
        }
        self.debug_logs.append(log_entry)

        if self.DEBUG or self.DEEP_DEBUG:
            print(f"[{level}] {method}: {message}")
            if data and self.DEEP_DEBUG:
                print(f"    Данные: {data}")

    def _log_response(self, stage, raw_response, parsed, expected=None, metadata=None):
        if not self.DEEP_DEBUG:
            return

        print(f"\n{'='*80}")
        print(f"[DEEP_DEBUG] {stage}")
        print(f"Сырой ответ ({len(raw_response)} chars):")
        print(f"{raw_response[:500]}...")
        print(f"Распарсено: {parsed}")
        if expected is not None:
            print(f"Ожидалось: {expected}")
            print(f"Совпадение: {parsed == expected}")
        if metadata:
            print(f"Метаданные: {metadata}")
        print(f"{'='*80}\n")

        log_entry = {
            "timestamp": time.time(),
            "stage": stage,
            "raw_response": raw_response[:1000],
            "parsed": parsed,
            "expected": expected,
            "metadata": metadata
        }
        self.debug_logs.append(log_entry)

    def _options_parser(self, options):
        self._log("_options_parser", "начало", {
            "raw_input": str(options)[:200],
            "type": type(options)
        }, "DEEP_DEBUG")


        if isinstance(options, list):
            self._log("_options_parser", "уже список", {"len": len(options), "first_3": options[:3]}, "DEEP_DEBUG")
            return options

        original_input = str(options)


        if isinstance(options, str):

            text = original_input.strip()


            if text.startswith('[') and text.endswith(']'):
                try:

                    json_text = text.replace("'", '"')
                    parsed = json.loads(json_text)
                    if isinstance(parsed, list):
                        self._log("_options_parser", "JSON парсинг успешен", {"len": len(parsed)}, "DEBUG")
                        return parsed
                except json.JSONDecodeError as e:
                    self._log("_options_parser", "JSON ошибка", {"error": str(e)}, "DEBUG")


            if text.startswith('[') and text.endswith(']'):

                content = text[1:-1].strip()
                self._log("_options_parser", "формат с пробелами", {"content_preview": content[:100]}, "DEEP_DEBUG")

                items = []
                current_item = ""
                in_quotes = False
                quote_char = None

                i = 0
                while i < len(content):
                    char = content[i]

                    if char in ['"', "'"]:
                        if not in_quotes:

                            in_quotes = True
                            quote_char = char
                            current_item += char
                        elif char == quote_char:

                            in_quotes = False
                            current_item += char
                            items.append(current_item)
                            current_item = ""


                            i += 1
                            while i < len(content) and content[i] in [' ', '\n', '\t']:
                                i += 1
                            continue
                        else:
                            current_item += char
                    elif char == ' ' and not in_quotes:

                        if current_item:
                            items.append(current_item)
                            current_item = ""
                    else:
                        current_item += char

                    i += 1


                if current_item:
                    items.append(current_item)


                cleaned_items = []
                for item in items:
                    item = item.strip()
                    if item:
                        if (item.startswith('"') and item.endswith('"')) or \
                        (item.startswith("'") and item.endswith("'")):
                            item = item[1:-1]

                        item = item.replace('\\"', '"').replace("\\'", "'").replace('\\n', '\n')
                        cleaned_items.append(item)

                if cleaned_items:
                    self._log("_options_parser", "специальный формат распарсен", {
                        "count": len(cleaned_items),
                        "first_3": cleaned_items[:3]
                    }, "DEBUG")
                    return cleaned_items


        if isinstance(options, str) and ',' in options:
            try:

                parts = []
                current = ""
                in_quotes = False
                quote_char = None

                for char in options:
                    if char in ['"', "'"]:
                        if not in_quotes:
                            in_quotes = True
                            quote_char = char
                        elif char == quote_char:
                            in_quotes = False
                        current += char
                    elif char == ',' and not in_quotes:
                        parts.append(current.strip())
                        current = ""
                    else:
                        current += char

                if current:
                    parts.append(current.strip())


                cleaned_parts = []
                for part in parts:
                    part = part.strip()
                    if part:
                        if (part.startswith('"') and part.endswith('"')) or \
                        (part.startswith("'") and part.endswith("'")):
                            part = part[1:-1]
                        cleaned_parts.append(part)

                if cleaned_parts:
                    self._log("_options_parser", "разделили по запятым", {
                        "count": len(cleaned_parts),
                        "first_3": cleaned_parts[:3]
                    }, "DEBUG")
                    return cleaned_parts
            except Exception as e:
                self._log("_options_parser", "ошибка при разделении по запятым", {"error": str(e)}, "DEBUG")


        if isinstance(options, str) and len(options) > 10:
            self._log("_options_parser", "пробуем LLM парсинг", None, "DEBUG")
            llm_parsed = self._llm_parse_options(options)
            if llm_parsed:
                return llm_parsed


        self._log("_options_parser", "не удалось распарсить", {
            "original_length": len(original_input),
            "original_preview": original_input[:200]
        }, "DEBUG")

        return ["Варианты не предоставлены"]

    def _llm_parse_options(self, options_text):
        """Использование LLM для парсинга сложных форматов опций"""
        try:
            system_prompt = """Ты ассистент для парсинга данных. Извлеки список вариантов ответа из текста.
    Текст может быть в разных форматах: JSON, Python список, или строковое представление.
    Верни ТОЛЬКО валидный JSON список строк.

    Пример 1:
    Вход: "['Вариант A' 'Вариант B' 'Вариант C']"
    Выход: ["Вариант A", "Вариант B", "Вариант C"]

    Пример 2:
    Вход: "['Южная Америка' 'Южная Азия' 'Северная Африка']"
    Выход: ["Южная Америка", "Южная Азия", "Северная Африка"]

    Пример 3:
    Вход: "['Верно, Неверно' 'Не указано, Не указано']"
    Выход: ["Верно, Неверно", "Не указано, Не указано"]

    ПРАВИЛА:
    1. Всегда возвращай валидный JSON
    2. Только список строк
    3. Сохраняй оригинальный текст вариантов
    4. Если не можешь распарсить - верни пустой список []"""

            user_prompt = f"""Извлеки список вариантов из текста:

    Текст: {options_text}

    JSON список:"""

            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]

            text = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            inputs = self.tokenizer(text, return_tensors="pt").to(self.device)

            with torch.no_grad():
                generated_ids = self.model.generate(
                    inputs.input_ids,
                    attention_mask=inputs.attention_mask,
                    max_new_tokens=500,
                    temperature=0.1,
                    do_sample=False,
                    pad_token_id=self.tokenizer.pad_token_id
                )

            llm_response = self.tokenizer.decode(
                generated_ids[0][inputs.input_ids.shape[1]:],
                skip_special_tokens=True
            )

            self._log("_llm_parse_options", "LLM ответ", {"response": llm_response[:200]}, "DEBUG")


            try:
                parsed = json.loads(llm_response)
                if isinstance(parsed, list):
                    self._log("_llm_parse_options", "успешно распарсено", {"count": len(parsed)}, "DEBUG")
                    return parsed
            except json.JSONDecodeError:

                import re
                json_match = re.search(r'\[.*\]', llm_response, re.DOTALL)
                if json_match:
                    try:
                        parsed = json.loads(json_match.group())
                        if isinstance(parsed, list):
                            self._log("_llm_parse_options", "нашли JSON в ответе", {"count": len(parsed)}, "DEBUG")
                            return parsed
                    except:
                        pass

            return []

        except Exception as e:
            self._log("_llm_parse_options", "ошибка", {"error": str(e)}, "DEBUG")
            return []

    def generate_cot_instruction(
        self,
        question: str,
        topic: str,
        encoded_options,
        max_tokens: int = 1000
    ) -> str:
        """
        Генерирует только CoT инструкцию для конкретного вопроса.

        Returns:
            str: Сгенерированная CoT инструкция или пустая строка при ошибке
        """
        self._log("generate_cot_instruction", "начало", {
            "topic": topic,
            "question_len": len(question),
            "max_tokens": max_tokens
        }, "DEBUG")

        options = self._options_parser(encoded_options)
        if not options or (len(options) == 1 and options[0] == "Варианты не предоставлены"):
            self._log("generate_cot_instruction", "нет опций", None, "DEBUG")
            return ""

        options_text = "\n".join([f"{i}. {opt}" for i, opt in enumerate(options)])

        system_prompt = """Ты эксперт по методике Chain of Thought (CoT).
    Создай структурированную пошаговую инструкцию для решения ЗАДАЧИ.

    ТРЕБОВАНИЯ:
    1. Инструкция должна быть КОНКРЕТНОЙ для данной задачи
    2. Используй нумерованные шаги (1., 2., 3., ...)
    3. Включи логические проверки и анализ вариантов
    4. Учитывай специфику категории задачи
    5. Заверши четким указанием формата ответа

    ФОРМАТ ВЫВОДА:
    Начни сразу с пошаговой инструкции, без вступлений."""

        user_prompt = f"""Создай Chain of Thought инструкцию для решения:

    КАТЕГОРИЯ: {topic}

    ВОПРОС:
    {question}

    ВАРИАНТЫ ОТВЕТОВ (всего {len(options)}):
    {options_text}

    Инструкция должна помочь систематически решить эту задачу.
    Индексы вариантов: от 0 до {len(options)-1}.

    CoT инструкция:"""

        try:
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]

            text = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            inputs = self.tokenizer(text, return_tensors="pt").to(self.device)

            with torch.no_grad():
                generated_ids = self.model.generate(
                    inputs.input_ids,
                    attention_mask=inputs.attention_mask,
                    max_new_tokens=max_tokens,
                    temperature=0.3,
                    do_sample=False,
                    pad_token_id=self.tokenizer.pad_token_id,
                )

            cot_instruction = self.tokenizer.decode(
                generated_ids[0][inputs.input_ids.shape[1]:],
                skip_special_tokens=True
            ).strip()

            self._log("generate_cot_instruction", "CoT сгенерирована", {
                "length": len(cot_instruction),
                "preview": cot_instruction[:200]
            }, "DEBUG")

            if self.DEEP_DEBUG:
                print(f"\n[DEEP_DEBUG] Сгенерированная CoT инструкция:")
                print(f"{cot_instruction}")
                print("-" * 80)

            return cot_instruction

        except Exception as e:
            self._log("generate_cot_instruction", "ошибка генерации", {"error": str(e)}, "DEBUG")
            return ""

    def generate_contextual_few_shot(
        self,
        question: str,
        topic: str,
        encoded_options,
        num_examples: int = 2,
        max_tokens_per_example: int = 1000
    ) -> str:
        """
        Генерирует few-shot примеры, тематически смежные с заданным вопросом.

        Args:
            question: Оригинальный вопрос (для определения тематики и сложности)
            topic: Категория вопроса
            encoded_options: Варианты ответов оригинального вопроса
            num_examples: Количество примеров для генерации
            max_tokens_per_example: Максимальное количество токенов на пример

        Returns:
            str: Сгенерированные few-shot примеры в формате строки
        """
        self._log("generate_contextual_few_shot", "начало", {
            "topic": topic,
            "num_examples": num_examples,
            "question_preview": question[:200]
        }, "DEBUG")

        options = self._options_parser(encoded_options)
        num_options = len(options) if options and options[0] != "Варианты не предоставлены" else 4

        question_complexity = "средней сложности" if len(question) > 100 else "базовой сложности"
        options_complexity = f"{num_options} вариантов" if num_options > 4 else "стандартное количество вариантов"

        system_prompt = f"""Ты эксперт по созданию few-shot инструкций для LLM по предмету "{topic}"
    Твоя задача - создать тематически смежные учебные примеры.

    ТРЕБОВАНИЯ К ПРИМЕРАМ:
    1. ТЕМАТИЧЕСКАЯ СМЕЖНОСТЬ: Примеры должны быть из той же узкой тематики, что и оригинальный вопрос
    2. СЛОЖНОСТЬ: Примеры должны быть примерно той же сложности
    3. ФОРМАТ: Каждый пример должен содержать:
    - Реалистичный вопрос по теме
    - Варианты ответов (от {max(3, num_options-2)} до {min(10, num_options+2)} вариантов)
    - Правильный ответ с индексом (ровно один)
    4. СТРУКТУРА: Один пример = один блок с четкой структурой

    ФОРМАТ ВЫВОДА для каждого примера:
    Пример [N]:
    Вопрос: [текст вопроса]
    Варианты ответа:
    0. [вариант 0]
    1. [вариант 1]
    ...
    [N]. [вариант N]
    Ответ: [индекс]

    Не пиши ничего кроме примеров."""

        question_analysis = f"""
    ОРИГИНАЛЬНЫЙ ВОПРОС (для анализа тематики и сложности):
    {question}

    ОПИСАНИЕ ОРИГИНАЛА:
    - Категория: {topic}
    - Сложность: {question_complexity}
    - Количество вариантов: {options_complexity}
    - Примерная тематика: {self._extract_topic_keywords(question, topic)}
    """

        user_prompt = f"""{question_analysis}

    Создай {num_examples} тематически смежных примера для few-shot обучения.

    УЧТИ:
    1. Тематика должна быть максимально близкой к оригинальному вопросу
    2. Сложность примеров должна быть сопоставимой
    3. Вариантов ответа: примерно {num_options} (от {max(3, num_options-2)} до {min(10, num_options+2)})
    4. Вопросы должны быть реалистичными и соответствовать предметной области
    5. Правильные ответы должны быть однозначными и проверяемыми

    Начни генерировать примеры:"""

        try:
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]

            text = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            inputs = self.tokenizer(text, return_tensors="pt").to(self.device)

            with torch.no_grad():
                generated_ids = self.model.generate(
                    inputs.input_ids,
                    attention_mask=inputs.attention_mask,
                    max_new_tokens=max_tokens_per_example * num_examples,
                    temperature=0.4,
                    do_sample=True,
                    top_p=0.9,
                    pad_token_id=self.tokenizer.pad_token_id,
                )

            few_shot_examples = self.tokenizer.decode(
                generated_ids[0][inputs.input_ids.shape[1]:],
                skip_special_tokens=True
            ).strip()

            formatted_examples = self._validate_and_format_few_shot(few_shot_examples, topic, num_options)

            self._log("generate_contextual_few_shot", "примеры сгенерированы", {
                "original_topic": topic,
                "generated_length": len(formatted_examples),
                "num_examples_found": formatted_examples.count("Пример"),
                "preview": formatted_examples[:300]
            }, "DEBUG")

            if self.DEEP_DEBUG:
                print(f"\n[DEEP_DEBUG] Сгенерированные few-shot примеры для '{topic}':")
                print("-" * 80)
                print(formatted_examples)
                print("-" * 80)

            return formatted_examples

        except Exception as e:
            self._log("generate_contextual_few_shot", "ошибка генерации", {"error": str(e)}, "DEBUG")
            return ""

    def _extract_topic_keywords(self, question: str, topic: str) -> str:
        """Извлекает ключевые слова из вопроса для определения узкой тематики"""
        keywords = []

        if topic == 'math':
            math_terms = ['гомоморфизм', 'ядро', 'инъективный', 'кольцо', 'идеал', 'уравнение',
                        'производная', 'интеграл', 'матрица', 'вектор', 'теорема', 'доказательство']
            keywords = [term for term in math_terms if term in question.lower()]

        elif topic == 'physics':
            physics_terms = ['скорость', 'ускорение', 'сила', 'энергия', 'заряд', 'волна',
                            'температура', 'давление', 'оптика', 'механика', 'электричество']
            keywords = [term for term in physics_terms if term in question.lower()]

        elif topic == 'history':
            history_terms = ['год', 'век', 'война', 'революция', 'договор', 'император',
                            'сражение', 'реформа', 'хронология', 'период', 'событие']
            keywords = [term for term in history_terms if term in question.lower()]

        if not keywords:
            words = question.lower().split()
            stop_words = {'этот', 'вопрос', 'относится', 'следующей', 'информации', 'какой', 'что', 'как'}
            keywords = [w for w in words if len(w) > 4 and w not in stop_words][:5]

        return ", ".join(keywords[:3]) if keywords else "общая тематика"

    def _validate_and_format_few_shot(self, examples_text: str, topic: str, expected_options: int) -> str:
        """Проверяет и форматирует сгенерированные few-shot примеры"""
        lines = examples_text.strip().split('\n')
        formatted = []
        current_example = []

        for line in lines:
            line = line.strip()
            if not line:
                continue

            if line.startswith(('Пример', 'пример', 'EXAMPLE', 'example')):
                if current_example:
                    formatted_example = self._format_single_example(current_example, topic, expected_options)
                    if formatted_example:
                        formatted.append(formatted_example)
                    current_example = []
                current_example.append(line)
            elif current_example:
                current_example.append(line)

        if current_example:
            formatted_example = self._format_single_example(current_example, topic, expected_options)
            if formatted_example:
                formatted.append(formatted_example)

        if not formatted:
            fallback = self._create_fallback_examples(topic, expected_options)
            return fallback

        return "\n\n".join(formatted)

    def _format_single_example(self, example_lines: list, topic: str, expected_options: int) -> str:
        """Форматирует один few-shot пример"""
        example_text = "\n".join(example_lines)

        has_question = any('вопрос:' in line.lower() for line in example_lines)
        has_options = any('вариант' in line.lower() for line in example_lines)
        has_answer = any('ответ:' in line.lower() for line in example_lines)

        if not (has_question and has_options and has_answer):
            return ""

        option_lines = [line for line in example_lines if re.match(r'^\d+\.', line.strip())]
        num_options = len(option_lines)

        if abs(num_options - expected_options) > 3 and expected_options > 3:
            return ""
        return example_text

    def _create_fallback_examples(self, topic: str, num_options: int) -> str:
        """Создает fallback примеры если генерация не удалась"""
        self._log("_create_fallback_examples", "создаем fallback", {
            "topic": topic,
            "num_options": num_options
        }, "DEBUG")


        fallbacks = self.few_shot_prompts
        if topic in fallbacks.keys():
            return fallbacks[topic]

        return f"""Пример 1:
    Вопрос: Пример вопроса по теме {topic}
    Варианты ответа:
    0. Вариант A
    1. Вариант B
    2. Вариант C
    {'' if num_options <= 3 else '3. Вариант D\n' + '\n'.join([f'{i}. Вариант {chr(65+i)}' for i in range(4, min(8, num_options))])}
    Ответ: 0"""

    def llm_parse_answer(self, raw_response: str) -> int:
        self._log("llm_parse_answer", "начало", {"raw_len": len(raw_response)}, "DEEP_DEBUG")

        system_prompt = """Ты - ассистент по извлечению чисел. Извлеки ЧИСЛО из текста.
Пример 1:
Текст: "Ответ: 2. Этот вариант правильный потому что..."
Извлеченное число: 2

Пример 2:
Текст: "Я считаю, что правильный вариант третий"
Извлеченное число: 2

Пример 3:
Текст: "Вариант А кажется верным"
Извлеченное число: 0

Пример 4:
Текст: "Номер правильного ответа: 5"
Извлеченное число: 4

ПРАВИЛА:
1. Извлекай ТОЛЬКО число (0, 1, 2, 3, ...)
2. Если в тексте несколько чисел - берем первое
3. Если указана буква (A=0, B=1, C=2, D=3)
4. Если не можешь извлечь - возвращай 0
5. Только число, без текста"""

        user_prompt = f"""Извлеки число из текста:

Текст: {raw_response}

Извлеченное число:"""

        try:
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]

            text = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            inputs = self.tokenizer(text, return_tensors="pt").to(self.device)

            with torch.no_grad():
                generated_ids = self.model.generate(
                    inputs.input_ids,
                    attention_mask=inputs.attention_mask,
                    max_new_tokens=300,
                    temperature=0.1,
                    do_sample=False,
                    pad_token_id=self.tokenizer.pad_token_id
                )

            llm_parsed = self.tokenizer.decode(
                generated_ids[0][inputs.input_ids.shape[1]:],
                skip_special_tokens=True
            )

            self._log("llm_parse_answer", "LLM ответ", {"llm_parsed": llm_parsed}, "DEEP_DEBUG")

            regex_parsed = self._regex_parse_answer(llm_parsed)

            self._log("llm_parse_answer", "финальный результат", {
                "raw_response_len": len(raw_response),
                "llm_parsed": llm_parsed,
                "regex_parsed": regex_parsed
            }, "DEBUG")

            return regex_parsed

        except Exception as e:
            self._log("llm_parse_answer", "ошибка", {"error": str(e)}, "DEBUG")
            return 0

    def _regex_parse_answer(self, text: str) -> int:
        import re

        text = str(text).strip()

        if not text:
            return 0

        try:
            num = int(text)
            return num
        except:
            pass

        patterns = [
            r'^\s*(\d+)\s*$',
            r'ответ[:\s]*(\d+)',
            r'число[:\s]*(\d+)',
            r'номер[:\s]*(\d+)',
            r'\b(\d+)\b',
        ]

        for pattern in patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                try:
                    num = int(match.group(1))
                    return num
                except:
                    continue

        letter_match = re.search(r'\b([a-d])\b', text, re.IGNORECASE)
        if letter_match:
            letter = letter_match.group(1).upper()
            return ord(letter) - ord('A')

        return 0

    def parse_answer_index(self, answer_text: str, use_llm_parsing=True) -> int:
        self._log("parse_answer_index", "начало", {"raw": str(answer_text)[:100], "use_llm_parsing": use_llm_parsing}, "DEEP_DEBUG")

        if not answer_text:
            self._log("parse_answer_index", "пустой ответ", None, "DEBUG")
            return 0

        answer_text = str(answer_text).strip()

        try:
            num = int(answer_text)
            self._log("parse_answer_index", "прямое число", {"num": num}, "DEBUG")
            return num
        except:
            pass

        if use_llm_parsing and len(answer_text) > 10:
            llm_result = self.llm_parse_answer(answer_text)
            self._log("parse_answer_index", "LLM парсинг", {"llm_result": llm_result}, "DEBUG")

            if llm_result != 0:
                return llm_result

        regex_result = self._regex_parse_answer(answer_text)
        self._log("parse_answer_index", "регулярки", {"regex_result": regex_result}, "DEBUG")

        return regex_result

    def direct_prompt(
        self,
        user_prompt:str,
        system_prompt:str,
        tokens:int = 1000,
        temperature:float = 0.1,
        few_shot = True
    ):
        self._log("direct_prompt", "начало", {
            "user_len": len(user_prompt),
            "system_len": len(system_prompt),
            "temperature": temperature
        }, "DEEP_DEBUG")

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]

        text = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

        self._log("direct_prompt", "шаблон применен", {"input_length": len(text)}, "DEEP_DEBUG")

        inputs = self.tokenizer(text, return_tensors="pt").to(self.device)

        with torch.no_grad():
            generated_ids = self.model.generate(
                inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=tokens,
                temperature=temperature,
                do_sample=False,
                pad_token_id=self.tokenizer.pad_token_id
            )

        response = self.tokenizer.decode(
            generated_ids[0][inputs.input_ids.shape[1]:],
            skip_special_tokens=True
        )

        self._log("direct_prompt", "получен ответ", {"response_length": len(response)}, "DEBUG")

        if self.DEEP_DEBUG:
            print(f"[DEEP_DEBUG] direct_prompt response ({len(response)} chars):")
            print(f"{response[:500]}...")

        return response

    def get_debug_logs(self):
        return self.debug_logs

    def clear_debug_logs(self):
        self.debug_logs = []

    def save_debug_logs(self, filename="llm_debug_logs.json"):
        import json
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(self.debug_logs, f, ensure_ascii=False, indent=2)
        print(f"Логи сохранены в {filename}")

    def print_debug_summary(self):
        if not self.DEBUG and not self.DEEP_DEBUG:
            print("Отладка отключена")
            return

        print(f"\n{'='*60}")
        print("СВОДКА ОТЛАДКИ LLM")
        print(f"{'='*60}")
        print(f"Всего логов: {len(self.debug_logs)}")

        if self.debug_logs:
            methods = {}
            levels = {}
            for log in self.debug_logs:
                method = log.get("method", "unknown")
                level = log.get("level", "unknown")
                methods[method] = methods.get(method, 0) + 1
                levels[level] = levels.get(level, 0) + 1

            print("\nВызовы методов:")
            for method, count in sorted(methods.items()):
                print(f"  {method}: {count}")

            print("\nУровни логирования:")
            for level, count in sorted(levels.items()):
                print(f"  {level}: {count}")

            print(f"\nDEBUG: {self.DEBUG}")
            print(f"DEEP_DEBUG: {self.DEEP_DEBUG}")


    def _calculate_metrics(
        self,
        results_df,
        answer_column,
        category_results,
        processing_times,
        method
    ):
        prediction_column = 'answer' if 'answer' in results_df.columns else 'predicted'
        
        def convert_to_json_serializable(obj):
            if isinstance(obj, (np.integer, int)):
                return int(obj)
            elif isinstance(obj, (np.floating, float)):
                return float(obj)
            elif isinstance(obj, (np.bool_, bool)):
                return bool(obj)
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            elif isinstance(obj, dict):
                return {k: convert_to_json_serializable(v) for k, v in obj.items()}
            elif isinstance(obj, list):
                return [convert_to_json_serializable(item) for item in obj]
            else:
                return obj
        
        metrics = {
            'method': method,
            'total_questions': len(results_df),
            'avg_processing_time': np.mean(processing_times).item() if processing_times else 0.0,
            'total_processing_time': sum(processing_times)
        }
        
        has_real_answers = False
        if answer_column is not None and answer_column in results_df.columns:
            has_real_answers = results_df[answer_column].notna().any().item()
        
        if has_real_answers and 'is_correct' in results_df.columns:
            correct = results_df['is_correct'].sum().item()
            accuracy = (correct / len(results_df)).item() if len(results_df) > 0 else 0.0

            metrics.update({
                'correct_answers': correct,
                'accuracy': accuracy,
                'accuracy_percent': f"{accuracy * 100:.2f}%",
                'error_rate': (1 - accuracy).item()
            })

            category_metrics = {}
            for category, results in category_results.items():
                if results['truths']:
                    cat_total = len(results['truths'])
                    cat_correct = results['correct']
                    cat_accuracy = (cat_correct / cat_total) if cat_total > 0 else 0.0

                    category_metrics[category] = {
                        'total': cat_total,
                        'correct': cat_correct,
                        'accuracy': cat_accuracy,
                        'accuracy_percent': f"{cat_accuracy * 100:.2f}%"
                    }
                else:
                    category_metrics[category] = {
                        'total': results['total'],
                        'correct': 0,
                        'accuracy': 0.0,
                        'accuracy_percent': "0.00%",
                        'note': 'no ground truth answers'
                    }

            metrics['category_metrics'] = category_metrics
            
            max_classes = 10
            confusion = np.zeros((max_classes, max_classes), dtype=int)
            answer_count = 0
            
            for idx, row in results_df.iterrows():
                if pd.notna(row[answer_column]) and pd.notna(row[prediction_column]):
                    answer_count += 1
                    try:
                        true_val = self.parse_answer_index(str(row[answer_column]))
                    except:
                        true_val = 0
                    try:
                        pred_val = self.parse_answer_index(str(row[prediction_column]))
                    except:
                        pred_val = 0

                    if 0 <= true_val < max_classes and 0 <= pred_val < max_classes:
                        confusion[true_val][pred_val] += 1
            
            if answer_count > 0:
                metrics['confusion_matrix'] = confusion.tolist()

                per_class_accuracy = []
                for i in range(max_classes):
                    total_class = sum(confusion[i]).item()
                    if total_class > 0:
                        class_acc = (confusion[i][i] / total_class).item()
                        per_class_accuracy.append({
                            'class': i,
                            'correct': confusion[i][i].item(),
                            'total': total_class,
                            'accuracy': class_acc,
                            'accuracy_percent': f"{class_acc * 100:.2f}%"
                        })
                    else:
                        per_class_accuracy.append({
                            'class': i,
                            'correct': 0,
                            'total': 0,
                            'accuracy': 0.0,
                            'accuracy_percent': "0.00%",
                            'note': 'no instances'
                        })

                metrics['per_class_accuracy'] = per_class_accuracy
        else:
            metrics['note'] = 'no ground truth answers provided'
            category_metrics = {}
            for category, results in category_results.items():
                category_metrics[category] = {
                    'total': results['total'],
                    'predictions_made': len(results['predictions']),
                    'has_ground_truth': bool(len(results['truths']) > 0)
                }
            metrics['category_metrics'] = category_metrics
        
        return convert_to_json_serializable(metrics)

    def process_csv_files(
        self,
        questions_csv_path: str,
        answers_csv_path: Optional[str] = None,
        output_dir: str = "./results",
        output_filename: Optional[str] = None,
        method: Literal[
            "ensemble_vote",
            "generate_answer",
            "confidence_ensemble_vote",
            "generate_answer_selfcheck"
        ] = "ensemble_vote",
        save_intermediate: bool = True,
        **method_kwargs
    ) -> Tuple[pd.DataFrame, List[str]]:

        self._log("process_csv_files", "начало обработки", {
            "questions_file": questions_csv_path,
            "answers_file": answers_csv_path,
            "output_dir": output_dir,
            "method": method
        }, "DEBUG")

        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)
        print(f"📁 Директория результатов: {output_path.absolute()}")

        print("📥 Загрузка данных...")
        try:
            questions_df = pd.read_csv(questions_csv_path)
            unnamed_cols = [col for col in questions_df.columns if 'Unnamed' in str(col)]
            if unnamed_cols:
                questions_df = questions_df.rename(columns={unnamed_cols[0]: 'question_id'})
                print(f"  ✓ Безымянная колонка переименована в 'question_id'")

            required_cols = ['question', 'options', 'category']
            missing_cols = [col for col in required_cols if col not in questions_df.columns]

            if missing_cols:
                raise ValueError(f"Отсутствуют необходимые колонки: {missing_cols}")

            questions_df['question_id'] = questions_df['question_id'].astype(int)
            questions_df['question'] = questions_df['question'].astype(str)
            questions_df['options'] = questions_df['options'].astype(str)
            questions_df['category'] = questions_df['category'].astype(str)

            print(f"  ✓ Загружено {len(questions_df)} вопросов")
            print(f"  ✓ Категории: {questions_df['category'].unique()[:5]}")

        except Exception as e:
            self._log("process_csv_files", "ошибка загрузки вопросов", {"error": str(e)}, "DEBUG")
            raise ValueError(f"Ошибка загрузки файла с вопросами: {e}")

        answers_df = None
        answers_array = [] 

        if answers_csv_path:
            try:
                answers_df = pd.read_csv(answers_csv_path)
                unnamed_cols = [col for col in answers_df.columns if 'Unnamed' in str(col)]
                if unnamed_cols:
                    answers_df = answers_df.rename(columns={unnamed_cols[0]: 'question_id'})

                if 'answer' not in answers_df.columns:
                    other_cols = [col for col in answers_df.columns if col != 'question_id']
                    if len(other_cols) == 1:
                        answers_df = answers_df.rename(columns={other_cols[0]: 'true_answer'})
                    else:
                        raise ValueError("Не могу найти колонку 'answer' в файле с ответами")
                
                answers_df['question_id'] = answers_df['question_id'].astype(int)
                answers_df['true_answer'] = answers_df['true_answer'].astype(str)
                answers_array = answers_df['true_answer'].tolist()
                print(f"  ✓ Загружено {len(answers_df)} ответов")

                q_ids = set(questions_df['question_id'])
                a_ids = set(answers_df['question_id'])

                if q_ids != a_ids:
                    missing_in_answers = q_ids - a_ids
                    missing_in_questions = a_ids - q_ids

                    if missing_in_answers:
                        print(f"  ⚠️  ВНИМАНИЕ: Ответы отсутствуют для вопросов: {sorted(missing_in_answers)[:10]}")
                    if missing_in_questions:
                        print(f"  ⚠️  ВНИМАНИЕ: Лишние ответы для вопросов: {sorted(missing_in_questions)[:10]}")

            except Exception as e:
                self._log("process_csv_files", "ошибка загрузки ответов", {"error": str(e)}, "DEBUG")
                print(f"  ⚠️  Предупреждение: не удалось загрузить ответы: {e}")
                answers_df = None
                answers_array = []

        if answers_df is not None:
            merged_df = pd.merge(
                questions_df,
                answers_df[['question_id', 'true_answer']],
                on='question_id',
                how='left'
            )
            merged_df['has_answer'] = merged_df['true_answer'].notna()
            print(f"  ✓ Объединено: {merged_df['has_answer'].sum()} вопросов с ответами, "
                f"{len(merged_df) - merged_df['has_answer'].sum()} без ответов")
        else:
            merged_df = questions_df.copy()
            merged_df['true_answer'] = None
            merged_df['has_answer'] = False
            print("  ✓ Ответы не предоставлены, будут сохранены только предсказания")

        print(f"\n🤖 Запуск модели (метод: {method})...")
        
        answer_column = "true_answer" if 'true_answer' in merged_df.columns else None
        eval_kwargs = {
            "question_column": "question",
            "options_column": "options",
            "category_column": "category",
            "answer_column": answer_column,
            "method": method,
            "method_kwargs": method_kwargs
        }

        eval_kwargs = {k: v for k, v in eval_kwargs.items() if v is not None}

        if save_intermediate:
            intermediate_path = output_path / "intermediate_data.csv"
            merged_df.to_csv(intermediate_path, index=False)
            print(f"  💾 Промежуточные данные сохранены: {intermediate_path}")

        try:
            results_df, metrics = self.evaluate_dataframe(
                merged_df,
                **eval_kwargs
            )

            print("\n✅ Обработка завершена!")
            print(f"📊 Обработано вопросов: {len(results_df)}")
            
            if answer_column and merged_df[answer_column].notna().any():
                if 'is_correct' in results_df.columns:
                    correct = results_df['is_correct'].sum()
                    accuracy = correct / len(results_df) if len(results_df) > 0 else 0
                    print(f"🎯 Точность на тестовых данных: {correct}/{len(results_df)} ({accuracy:.2%})")
                else:
                    print("ℹ️  Колонка 'is_correct' не создана")
            else:
                print("ℹ️  Ответы для проверки не предоставлены, точность не рассчитывается")
        
        except Exception as e:
            self._log("process_csv_files", "ошибка при оценке", {"error": str(e)}, "DEBUG")
            if save_intermediate and intermediate_path.exists():
                print("🔄 Загружаю промежуточные данные после ошибки...")
                results_df = pd.read_csv(intermediate_path)
                results_df['predicted'] = 0
                results_df['is_correct'] = False
                metrics = {'error': str(e)}
            else:
                raise RuntimeError(f"Ошибка при обработке данных: {e}")

        print("\n💾 Сохранение результатов...")
        if output_filename is None:
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            model_name_safe = self.model_name.replace("/", "_")
            output_filename = f"results_{model_name_safe}_{timestamp}.csv"

        output_filepath = output_path / output_filename
        
        results_df = results_df.rename(columns={'predicted': 'answer'})
        
        final_columns = [
            'question_id', 'question', 'category',
            'answer',
            'processing_time'
        ]
        
        if 'true_answer' in results_df.columns and results_df['true_answer'].notna().any():
            final_columns.append('true_answer')
        
        if 'options' in results_df.columns:
            results_df['options_preview'] = results_df['options'].apply(
                lambda x: str(x)[:200] + "..." if len(str(x)) > 200 else str(x)
            )
            final_columns.append('options_preview')
        
        if 'is_correct' in results_df.columns and results_df['is_correct'].notna().any():
            final_columns.append('is_correct')
        
        final_df = results_df[final_columns].copy()

        try:
            final_df.to_csv(output_filepath, index=False, encoding='utf-8-sig')
            import os
            if os.path.exists(output_filepath):
                file_size = os.path.getsize(output_filepath)
                print(f"  ✅ Файл успешно создан: {output_filepath}")
                print(f"  📄 Размер файла: {file_size / 1024:.1f} KB ({file_size} байт)")
                try:
                    preview_df = pd.read_csv(output_filepath, nrows=3)
                    print("👀 Предпросмотр (первые 3 строки):")
                    print(preview_df.to_string(index=False))
                except Exception as e:
                    print(f"  👀 Файл создан, но не читается как CSV: {e}")
            else:
                print(f"  ❗ ФАЙЛ НЕ СОЗДАН! Проверь путь: {output_filepath}")
                
        except Exception as e:
            print(f"  ❗ Ошибка при сохранении файла: {e}")
            print("  ❗ Попробуй другой путь или проверь права доступа")

        if not answers_array:
            print("  ℹ️  Массив ответов пустой (файл с ответами не предоставлен или пуст)")

        if metrics and self.DEBUG:
            print("\n📊 МЕТРИКИ:")
            print("-" * 40)
            for key, value in metrics.items():
                if key not in ['confusion_matrix', 'per_class_accuracy', 'category_metrics']:
                    print(f"  {key}: {value}")
            print("-" * 40)


        print(f"\n{'='*60}")
        print("СВОДКА ОБРАБОТКИ")
        print(f"{'='*60}")
        print(f"Обработано вопросов: {len(final_df)}")
        print(f"Категории: {', '.join(sorted(final_df['category'].unique()))}")

        print(f"\n📊 ИНФОРМАЦИЯ О МАССИВЕ ОТВЕТОВ:")
        print(f"  Всего ответов в массиве: {len(answers_array)}")
        if answers_array:
            null_count = sum(1 for a in answers_array if pd.isna(a) or str(a).strip() == '' or str(a).lower() == 'none')
            print(f"  Пустых ответов: {null_count}")
            unique_answers = len(set(str(a) for a in answers_array if pd.notna(a)))
            print(f"  Уникальных ответов: {unique_answers}")
            
            print(f"\n📖 Первые 10 ответов:")
            print("-" * 40)
            for i, answer in enumerate(answers_array[:10]):
                answer_str = str(answer) if pd.notna(answer) else "[пусто]"
                print(f"{i+1:3d}. {answer_str[:100]}{'...' if len(answer_str) > 100 else ''}")
            print("-" * 40)
        else:
            print("  ℹ️  Массив ответов пустой (файл с ответами не предоставлен)")

        if 'is_correct' in final_df.columns and final_df['is_correct'].notna().any():
            accuracy = final_df['is_correct'].mean()
            print(f"\n🎯 Точность: {accuracy:.2%}")
            if len(final_df['category'].unique()) > 1:
                print("\n📈 Статистика по категориям:")
                for category in sorted(final_df['category'].unique()):
                    cat_df = final_df[final_df['category'] == category]
                    if len(cat_df) > 0 and 'is_correct' in cat_df.columns:
                        cat_acc = cat_df['is_correct'].mean()
                        print(f"  {category}: {cat_acc:.2%} ({len(cat_df)} вопросов)")
        else:
            print("\nℹ️  Точность не рассчитана (нет ответов для проверки)")

        avg_time = final_df['processing_time'].mean() if 'processing_time' in final_df.columns else 0
        print(f"\n⏱️  Среднее время на вопрос: {avg_time:.2f} секунд")
        print(f"💾 Файл результатов: {output_filepath}")
        print(f"{'='*60}")

        self._log("process_csv_files", "обработка завершена", {
            "output_file": str(output_filepath),
            "rows_processed": len(final_df),
            "answers_count": len(answers_array),
            "has_answers": answers_csv_path is not None
        }, "DEBUG")

        return final_df, answers_array

In [ ]:
exp = LLM(
    model=model_14b,
    tokenizer=tokenizer_14b,
    deep_debug=True,
    use_llm_parsing=True,
    use_selfcheck=False,
    llm_few_shot_generation=False,
    llm_cot_generation=True,
)

In [ ]:
exp.process_csv_files(
    questions_csv_path="/kaggle/input/LR1_dev.csv",
    answers_csv_path="/kaggle/input/LR1_dev_answers.csv",
    output_dir="/kaggle/working",
    method="generate_answer_selfcheck"
)

In [ ]:
exp.evaluate_dataframe(
    df=train_df,
    use_selfcheck=False,
    use_llm_parsing=True,
)

In [16]:
problematic_categories = [
    "health",
    "philosophy", 
    "physics",
    "law",
]

train_df_filtered = train_df[
    train_df['category'].isin(problematic_categories)
]

train_df_filtered.shape

(14, 5)

In [18]:
train_df_filtered.sort_values(
    by='category'
).head(14)

,id,question,options,category,answer
14,14,"49-летний мужчина, который восстанавливается в больнице через два дня после неосложненного шунтирования левой бедренно-подколенной артерии из-за клодикации, теперь испытывает усиливающуюся боль в левой стопе. До этого момента послеоперационный период пациента проходил без особых особенностей, и ему был назначен морфин в низких дозах для контроля боли. В анамнезе значится сахарный диабет 2 типа, контролируемый при помощи метформина и диеты. Сейчас жизненные показатели следующие: температура 36.8°C (98.2°F), пульс 80 уд/мин и регулярный, дыхание 20 вдохов/мин, артериальное давление 150/92 мм рт. ст. Хирургический разрез выглядит чистым и хорошо сомкнутым, без патологической покраснения или отека. Левая нижняя конечность и стопа выглядят бледными. При пальпации левой нижней конечности обнаруживается сильный бедренный пульс, слабый подколенный пульс и холодная стопа без пульса. Какой из следующих вариантов управления является наиболее подходящим?",['Интраоперационная ангиография' 'Внутривенная терапия гепарином'\n 'Немедленное хирургическое исследование' 'Начать антибиотикотерапию'\n 'Компьютерная томография (CT) нижней конечности'\n 'Интраартериальная терапия тканевым плазминогенным активатором (tPA)'\n 'Допплеровская ультрасонография левой нижней конечности'\n 'Магнитно-резонансная томография (MRI) левой нижней конечности'\n 'Измерение давления в отделениях на кровати' 'Обезболивание и наблюдение'],health,0
25,25,Трахея пересекается перешейком щитовидной железы на уровне,['гортаноглотки.' 'голосовых связок.' 'кольцевидного хряща.'\n 'подъязычной кости.' 'крикотиреоидной мембраны.' 'трахеальных колец.'\n 'миндалины.' 'эпиглоттиса.' 'голосовой щели.' 'щитовидного хряща.'],health,5
38,38,"Какое из следующих различий в возрасте может быть, частично, эффектом когорты? Различия в",['сердечно-сосудистых заболеваниях' 'алкоголизме' 'суициде' 'диабете'\n 'депрессии' 'шизофрении' 'деменции' 'астме' 'ожирении' 'гипертонии'],health,1
44,44,Сколько килокалорий содержится в одном грамме этанола?,['37.6 кДж или 9.0 ккал на г' '15.6 кДж или 3.7 ккал на г'\n '23.5 кДж или 5.8 ккал на г' '31.5 кДж или 7.5 ккал на г'\n '19.7 кДж или 4.1 ккал на г' '36.5 кДж или 8.1 ккал на г'\n '28.4 кДж или 6.8 ккал на г' '29.7 кДж или 7.1 ккал на г'\n '33.3 кДж или 7.9 ккал на г' '22.9 кДж или 5.5 ккал на г'],health,7
12,12,"Мужчина решил остановиться у автоматического киоска с гамбургерами для позднего перекуса. Когда он подъехал к очереди, менеджер киоска через интерком сообщил ему, что заведение закрывается и новые заказы не принимаются. Она сказала мужчине, что последний обслуженный автомобиль был прямо перед ним. Мужчина разозлился и закричал в интерком: ""Слушай, детка, я голоден. Я хочу два чизбургера, большую порцию картошки фри и Кока-Колу."" Менеджер ответила: ""Мне очень жаль, но мы не можем принять ваш заказ."" Вскоре после этого менеджер передала заказ пассажирам в машине перед мужчиной. Увидев, что менеджер обслуживает эту машину, мужчина сильно разозлился, подъехал к окну обслуживания и закричал на менеджера: ""Вы не можете так со мной поступать."" Когда менеджер засмеялась, мужчина внезапно достал из бардачка пистолет, нацелился на менеджера и выстрелил, собираясь попасть в неё. Пуля промахнулась мимо менеджера, но попала в клиента, ранив его в голову. В иске клиента против мужчины за причине...","['успешен, потому что мужчина причинил вред своими действиями, независимо от его намерений.'\n 'неуспешен, потому что мужчина не целился в клиента.'\n 'неуспешен, потому что менеджер могла предотвратить ситуацию, приняв заказ мужчины.'\n 'успешен, потому что мужчина действовал безрассудно и создал опасную ситуацию.'\n 'неуспешен, потому что мужчина не мог предвидеть, что пуля попадет в кого-то, кроме менеджера.'\n 'неуспешен, потому что мужчину спровоцировало отказ менеджера обслужить его.'\n 'неуспешен, потому что мужчина не намеревался стрелять в клиента.'\n 'успешен, потому что была ""существенная увереннос

In [ ]:
# Тест 1: Базовый ensemble_vote (как раньше, но улучшенный)
result_df1, metrics1 = exp.process_csv_files(
    questions_csv_path="/kaggle/input/llm-lab1/LR1.csv",
    output_dir="/kaggle/working",
    method="ensemble_vote",
    output_filename="results_ensemble_basic.csv",
    method_kwargs={
        "n_votes": 5,
        "temperatures": [0.1, 0.3, 0.5, 0.7, 0.9],
        "force_diversity": True
    }
)

# Тест 2: Ensemble с разнообразными промптами
result_df2, metrics2 = exp.process_csv_files(
    questions_csv_path="/kaggle/input/llm-lab1/LR1.csv",
    output_dir="/kaggle/working",
    method="ensemble_vote",
    output_filename="results_ensemble_diverse_prompts.csv",
    method_kwargs={
        "n_votes": 5,
        "temperatures": [0.1, 0.2, 0.3, 0.4, 0.5],
        "diverse_prompts": True,  # ВКЛЮЧАЕМ разные промпты
        "shuffle_options": True,  # Перемешиваем варианты
        "prompt_strategies": ["expert", "skeptic", "student", "analyst", "rush"]
    }
)

# Тест 3: Ensemble только с перемешиванием
result_df3, metrics3 = exp.process_csv_files(
    questions_csv_path="/kaggle/input/llm-lab1/LR1.csv",
    output_dir="/kaggle/working",
    method="ensemble_vote",
    output_filename="results_ensemble_shuffle_only.csv",
    method_kwargs={
        "n_votes": 3,
        "temperatures": [0.1, 0.5, 0.9],
        "shuffle_options": True,  # Только перемешивание
        "diverse_prompts": False
    }
)

In [ ]:
# Тест 4: Базовая проверка (normal)
result_df4, metrics4 = exp.process_csv_files(
    questions_csv_path="/kaggle/input/llm-lab1/LR1.csv",
    output_dir="/kaggle/working",
    method="generate_answer",  # Используем generate_answer с selfcheck
    output_filename="results_verification_normal.csv",
    method_kwargs={
        "use_selfcheck": True,  # Включаем встроенную проверку
        "temperature": 0.3
    }
)

# Тест 5: Глубокая проверка через ask_with_verification
result_df5, metrics5 = exp.process_csv_files(
    questions_csv_path="/kaggle/input/llm-lab1/LR1.csv",
    output_dir="/kaggle/working",
    method="ensemble_vote",  # Трюк: используем ensemble с 1 голосом + проверкой
    output_filename="results_deep_verification.csv",
    method_kwargs={
        "n_votes": 1,  # Только один голос
        "temperatures": [0.2],
        "use_selfcheck": False,
        # Кастомная проверка через ask_with_verification
        "custom_evaluator": lambda q, o, c: exp.ask_with_verification(
            question=q,
            encoded_options=o,
            category=c,
            temperature=0.2,
            verification_depth="deep",  # Глубокая проверка
            num_verification_rounds=2
        )[0]  # Берем только ответ, без веса
    }
)

# Тест 6: Агрессивная проверка (поиск ошибок)
result_df6, metrics6 = exp.process_csv_files(
    questions_csv_path="/kaggle/input/llm-lab1/LR1.csv",
    output_dir="/kaggle/working",
    method="generate_answer",
    output_filename="results_aggressive_verification.csv",
    method_kwargs={
        "use_selfcheck": False,
        "temperature": 0.1,
        "custom_postprocess": lambda response, question, options, category: 
            exp.ask_with_verification(
                question=question,
                encoded_options=options,
                category=category,
                temperature=0.1,
                verification_depth="aggressive",  # Агрессивная проверка
                weight_match=2.0,  # Высокий вес при совпадении
                weight_mismatch=0.3  # Низкий вес при несовпадении
            )[0]
    }
)

In [ ]:
# Тест 7: Базовый confidence_ensemble_vote
result_df7, metrics7 = exp.process_csv_files(
    questions_csv_path="/kaggle/input/llm-lab1/LR1.csv",
    output_dir="/kaggle/working",
    method="confidence_ensemble_vote",
    output_filename="results_confidence_basic.csv",
    method_kwargs={
        "n_runs": 4,
        "temperatures": [0.1, 0.2, 0.3, 0.4],
        "verification_depth": "normal"
    }
)

# Тест 8: Confidence с критической проверкой и арбитражем
result_df8, metrics8 = exp.process_csv_files(
    questions_csv_path="/kaggle/input/llm-lab1/LR1.csv",
    output_dir="/kaggle/working",
    method="confidence_ensemble_vote",
    output_filename="results_confidence_full.csv",
    method_kwargs={
        "n_runs": 5,
        "temperatures": [0.1, 0.25, 0.5, 0.75, 0.9],
        "use_critical_verification": True,  # ВКЛ критическую проверку
        "verification_depth": "deep",  # Глубокая проверка
        "arbitration_enabled": True,  # ВКЛ арбитраж
        "weight_calculation": "adaptive"  # Адаптивные веса
    }
)

# Тест 9: Confidence только с арбитражем (без критической проверки)
result_df9, metrics9 = exp.process_csv_files(
    questions_csv_path="/kaggle/input/llm-lab1/LR1.csv",
    output_dir="/kaggle/working",
    method="confidence_ensemble_vote",
    output_filename="results_confidence_arbitration_only.csv",
    method_kwargs={
        "n_runs": 3,
        "temperatures": [0.1, 0.5, 0.9],
        "use_critical_verification": False,  # ВЫКЛ критическую проверку
        "arbitration_enabled": True,  # Но арбитраж ВКЛ
        "weight_calculation": "confidence_based"  # Веса на основе уверенности
    }
)

In [ ]:
test = pd.read_csv(
    "res.csv"
)

In [29]:
test.columns

Index(['question_id', 'question', 'category', 'answer', 'processing_time',
       'options_preview'],
      dtype='object')

In [30]:
test = test[['answer']]
test.to_csv(
    "result.csv"
)

In [ ]:
import re

def extract_predictions_from_file(filepath: str) -> list[int]:
    """
    Читает txt файл с логами и извлекает все числа после 'предсказано='
    
    Args:
        filepath: Путь к txt файлу с логами
        
    Returns:
        Список извлеченных чисел (int)
    """
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            content = file.read()
        pattern = r'предсказано=(\d+)'
        matches = re.findall(pattern, content)
        predictions = [int(match) for match in matches]
        
        return predictions
        
    except FileNotFoundError:
        print(f"Ошибка: Файл '{filepath}' не найден.")
        return []
    except Exception as e:
        print(f"Ошибка при обработке файла: {e}")
        return []

res = extract_predictions_from_file(
    "da.txt"
)



In [39]:
res = [el for ind, el in enumerate(res) if ind%2==0]

[3, 8, 7, 3, 1, 1, 8, 4, 2, 6, 6, 6, 1, 2, 5, 0, 4, 0, 5, 3, 4, 9, 6, 4, 0, 1, 0, 8, 8, 3, 0, 2, 1, 3, 2, 0, 3, 1, 3, 4, 9, 4, 1, 5, 6, 2, 4, 6, 2, 8, 4, 8, 1, 4, 4, 9, 1, 1, 0, 2, 1, 3, 2, 8, 1, 2, 5, 4, 4, 6, 9, 7, 0, 0, 3, 0, 7, 2, 6, 1, 1, 0, 4, 0, 9, 3, 4, 5, 5, 4, 4, 2, 0, 3, 1, 9, 3, 4, 6, 5]


In [41]:
import csv

def create_csv_from_list(data: list, filename: str = "answers.csv"):
    """
    Создает CSV файл с колонкой answer из списка данных.
    
    Args:
        data: Список значений для колонки answer
        filename: Имя выходного CSV файла
    """
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            
            # Записываем заголовок
            writer.writerow(['answer'])
            
            # Записываем данные
            for item in data:
                writer.writerow([item])
        
        print(f"CSV файл '{filename}' успешно создан.")
        print(f"Создано {len(data)} записей.")
        
    except Exception as e:
        print(f"Ошибка при создании CSV файла: {e}")

create_csv_from_list(res)

CSV файл 'answers.csv' успешно создан.
Создано 100 записей.
